In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.tree import _tree
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import re
import random
import copy
import json

In [2]:
pd.set_option('display.max_rows', None)  # None means unlimited
pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.width', 1000)  # Adjust width to accommodate more columns

In [3]:
avg = pd.read_csv('../../source/processed_feature_averages.csv')

In [4]:
df = pd.read_csv('../../missing_feature/7_missing/dt_1_7missing_features.csv')

In [5]:
df.head()

,flow_duration,Header_Length,Protocol_Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot_sum,Min,Max,AVG,Std,Tot_size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,13.215984,-1.0,6.0,232.0,62.537910,62.537910,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,-1.0,-1.0,32.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2029.0,66.0,1072.0,-1.000000,-1.000000,66.0,-1.000000e+00,13.5,16.324726,354.298748,63225.672546,1.0,244.6,BenignTraffic
1,122.479718,62172.8,13.7,-1.0,5.057926,5.057926,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,172.8,351.2,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,1.0,-1.0,1101.9,50.0,230.0,71.085046,45.831857,68.1,1.665238e+08,13.5,11.920195,64.970328,2114.415628,1.0,244.6,BenignTraffic
2,17.803338,-1.0,7.1,83.1,81.390655,81.390655,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,1.6,-1.0,79.2,838.8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,1.0,1.0,17328.5,54.0,-1.0,-1.000000,827.989774,1441.5,1.665213e+08,13.5,47.082245,1170.507395,688084.298393,1.0,244.6,BenignTraffic
3,24.131660,1738870.0,6.0,64.0,70.136075,70.136075,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,2.0,0.0,64.1,1691.5,0.0,1.0,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,-1.0,-1.0,1.0,1.0,10183.0,-1.0,2672.4,-1.000000,505.204813,1777.2,2.089024e-04,5.5,60.131155,714.467498,320093.330391,0.8,38.5,BenignTraffic
4,-1.000000,575018.0,6.0,232.0,316.569854,316.569854,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,2.0,0.0,17.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,363.0,66.0,66.0,66.000000,0.000000,66.0,-1.000000e+00,5.5,11.489125,0.000000,-1.000000,0.0,38.5,BenignTraffic


In [6]:
# Convert the avg DataFrame into a dictionary
avg_dict = pd.Series(avg.Value.values, index=avg.Feature).to_dict()

# Replace -1 in df with the corresponding average value from avg_dict
for column in df.columns:
    if column in avg_dict:  # Ensure the column exists in avg_dict
        df[column] = df[column].apply(lambda x: avg_dict[column] if x == -1 else x)

In [7]:
print(avg_dict)

{'flow_duration': 178.34370165591196, 'Header_Length': 434899.5775208434, 'Protocol_Type': 11.770677389034304, 'Duration': 86.90195078389408, 'Rate': 4654.591609535896, 'Srate': 4654.591609535896, 'Drate': 2.0756022831703116e-06, 'ack_count': 0.0588487486607193, 'syn_count': 0.5437434884740976, 'fin_count': 0.0545546153846153, 'urg_count': 67.84796901784279, 'rst_count': 338.0698198724513, 'Tot_sum': 3879.447348788261, 'Min': 180.73943236376047, 'Max': 776.28682609752, 'AVG': 368.7064304867403, 'Std': 201.42974118021337, 'Tot_size': 370.1495738736666, 'IAT': 83522671.15636384, 'Number': 9.505953983023272, 'Magnitue': 22.443176233946197, 'Radius': 284.58945901339166, 'Covariance': 205955.5454897323, 'Variance': 0.5630919053833782, 'Weight': 141.70576443421632, 'fin_flag_number': 0.0, 'syn_flag_number': 0.0, 'rst_flag_number': 0.0, 'psh_flag_number': 0.0, 'ack_flag_number': 0.0, 'ece_flag_number': 0.0, 'cwr_flag_number': 0.0, 'HTTP': 0.0, 'HTTPS': 0.0, 'DNS': 0.0, 'Telnet': 0.0, 'SMTP': 

In [8]:
df.head()

,flow_duration,Header_Length,Protocol_Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot_sum,Min,Max,AVG,Std,Tot_size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,13.215984,4.348996e+05,6.0,232.000000,62.537910,62.537910,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.543743,0.054555,32.0,338.06982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2029.0,66.000000,1072.000000,368.706430,201.429741,66.0,8.352267e+07,13.5,16.324726,354.298748,63225.672546,1.0,244.6,BenignTraffic
1,122.479718,6.217280e+04,13.7,86.901951,5.057926,5.057926,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058849,0.000000,0.000000,172.8,351.20000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1101.9,50.000000,230.000000,71.085046,45.831857,68.1,1.665238e+08,13.5,11.920195,64.970328,2114.415628,1.0,244.6,BenignTraffic
2,17.803338,4.348996e+05,7.1,83.100000,81.390655,81.390655,0.000002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,1.600000,0.054555,79.2,838.80000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,17328.5,54.000000,776.286826,368.706430,827.989774,1441.5,1.665213e+08,13.5,47.082245,1170.507395,688084.298393,1.0,244.6,BenignTraffic
3,24.131660,1.738870e+06,6.0,64.000000,70.136075,70.136075,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,2.000000,0.000000,64.1,1691.50000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,10183.0,180.739432,2672.400000,368.706430,505.204813,1777.2,2.089024e-04,5.5,60.131155,714.467498,320093.330391,0.8,38.5,BenignTraffic
4,178.343702,5.750180e+05,6.0,232.000000,316.569854,316.569854,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,2.000000,0.000000,17.0,338.06982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,363.0,66.000000,66.000000,66.000000,0.000000,66.0,8.352267e+07,5.5,11.489125,0.000000,205955.545490,0.0,38.5,BenignTraffic


In [9]:
labels = ("BenignTraffic", "Brute_Force", "DDoS", "DoS", "Mirai", "Recon", "Spoofing", "Web-Based")

In [10]:
def predict_1(flow_duration, Header_Length, Protocol_Type, Duration, Rate, Srate, Drate, fin_flag_number, syn_flag_number, rst_flag_number, psh_flag_number, ack_flag_number, ece_flag_number, cwr_flag_number, ack_count, syn_count, fin_count, urg_count, rst_count, HTTP, HTTPS, DNS, Telnet, SMTP, SSH, IRC, TCP, UDP, DHCP, ARP, ICMP, IPv, LLC, Tot_sum, Min, Max, AVG, Std, Tot_size, IAT, Number, Magnitue, Radius, Covariance, Variance, Weight):
    dict = {}
    threshholds = []
    if Std <= 9.8:
        dict['Std'] = Std
        threshholds.append(['Std', '<=', 9.8])
        if IAT <= 83033340.0:
            dict['IAT'] = IAT
            threshholds.append(['IAT', '<=', 83033340.0])
            if rst_count <= 0.5:
                dict['rst_count'] = rst_count
                threshholds.append(['rst_count', '<=', 0.5])
                if Magnitue <= 10.63:
                    dict['Magnitue'] = Magnitue
                    threshholds.append(['Magnitue', '<=', 10.63])
                    if Tot_size <= 56.38:
                        dict['Tot_size'] = Tot_size
                        threshholds.append(['Tot_size', '<=', 56.38])
                        if Number <= 9.28:
                            dict['Number'] = Number
                            threshholds.append(['Number', '<=', 9.28])
                            if IAT <= 0.02:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 0.02])
                                if AVG <= 55.42:
                                    dict['AVG'] = AVG
                                    threshholds.append(['AVG', '<=', 55.42])
                                    if Radius <= 5.87:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 5.87])
                                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if Radius > 5.87
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 5.87])
                                        return [[0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ]], dict, threshholds
                                else:  # if AVG > 55.42
                                    dict['AVG'] = AVG
                                    threshholds.append(['AVG', '>', 55.42])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if IAT > 0.02
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 0.02])
                                if Number <= 7.01:
                                    dict['Number'] = Number
                                    threshholds.append(['Number', '<=', 7.01])
                                    if Tot_sum <= 303.1:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 303.1])
                                        return [[0.   , 0.57142857, 0.   , 0.   , 0.   , 0.   ,  0.14285714, 0.28571429]], dict, threshholds
                                    else:  # if Tot_sum > 303.1
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 303.1])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Number > 7.01
                                    dict['Number'] = Number
                                    threshholds.append(['Number', '>', 7.01])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if Number > 9.28
                            dict['Number'] = Number
                            threshholds.append(['Number', '>', 9.28])
                            if Number <= 9.56:
                                dict['Number'] = Number
                                threshholds.append(['Number', '<=', 9.56])
                                return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Number > 9.56
                                dict['Number'] = Number
                                threshholds.append(['Number', '>', 9.56])
                                return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if Tot_size > 56.38
                        dict['Tot_size'] = Tot_size
                        threshholds.append(['Tot_size', '>', 56.38])
                        if IAT <= 41462904.01:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 41462904.01])
                            if IAT <= 0.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 0.0])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if IAT > 0.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 0.0])
                                if IAT <= 0.01:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 0.01])
                                    return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if IAT > 0.01
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 0.01])
                                    if Radius <= 8.31:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 8.31])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if Radius > 8.31
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 8.31])
                                        return [[0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5]], dict, threshholds
                        else:  # if IAT > 41462904.01
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 41462904.01])
                            return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                else:  # if Magnitue > 10.63
                    dict['Magnitue'] = Magnitue
                    threshholds.append(['Magnitue', '>', 10.63])
                    if syn_count <= 0.29:
                        dict['syn_count'] = syn_count
                        threshholds.append(['syn_count', '<=', 0.29])
                        if flow_duration <= 0.39:
                            dict['flow_duration'] = flow_duration
                            threshholds.append(['flow_duration', '<=', 0.39])
                            if AVG <= 380.76:
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '<=', 380.76])
                                if Weight <= 77.83:
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '<=', 77.83])
                                    if AVG <= 59.13:
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '<=', 59.13])
                                        return [[0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5]], dict, threshholds
                                    else:  # if AVG > 59.13
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '>', 59.13])
                                        return [[0.   , 0.07692308, 0.   , 0.   , 0.   , 0.   ,  0.92307692, 0.   ]], dict, threshholds
                                else:  # if Weight > 77.83
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '>', 77.83])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if AVG > 380.76
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '>', 380.76])
                                if Duration <= 61.5:
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '<=', 61.5])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if Duration > 61.5
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '>', 61.5])
                                    return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                        else:  # if flow_duration > 0.39
                            dict['flow_duration'] = flow_duration
                            threshholds.append(['flow_duration', '>', 0.39])
                            if TCP <= 0.5:
                                dict['TCP'] = TCP
                                threshholds.append(['TCP', '<=', 0.5])
                                if Number <= 6.0:
                                    dict['Number'] = Number
                                    threshholds.append(['Number', '<=', 6.0])
                                    if Tot_sum <= 354.75:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 354.75])
                                        return [[0.16666667, 0.66666667, 0.   , 0.   , 0.   , 0.   ,  0.16666667, 0.   ]], dict, threshholds
                                    else:  # if Tot_sum > 354.75
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 354.75])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.02173913, 0.01086957,  0.86956522, 0.09782609]], dict, threshholds
                                else:  # if Number > 6.0
                                    dict['Number'] = Number
                                    threshholds.append(['Number', '>', 6.0])
                                    if Srate <= 695.91:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 695.91])
                                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if Srate > 695.91
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 695.91])
                                        return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                            else:  # if TCP > 0.5
                                dict['TCP'] = TCP
                                threshholds.append(['TCP', '>', 0.5])
                                return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if syn_count > 0.29
                        dict['syn_count'] = syn_count
                        threshholds.append(['syn_count', '>', 0.29])
                        if IAT <= 0.01:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 0.01])
                            if Covariance <= 90.4:
                                dict['Covariance'] = Covariance
                                threshholds.append(['Covariance', '<=', 90.4])
                                if flow_duration <= 38.82:
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '<=', 38.82])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if flow_duration > 38.82
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '>', 38.82])
                                    if UDP <= 0.5:
                                        dict['UDP'] = UDP
                                        threshholds.append(['UDP', '<=', 0.5])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if UDP > 0.5
                                        dict['UDP'] = UDP
                                        threshholds.append(['UDP', '>', 0.5])
                                        return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Covariance > 90.4
                                dict['Covariance'] = Covariance
                                threshholds.append(['Covariance', '>', 90.4])
                                if Covariance <= 97.33:
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '<=', 97.33])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if Covariance > 97.33
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '>', 97.33])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                        else:  # if IAT > 0.01
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 0.01])
                            if IAT <= 78690612.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 78690612.0])
                                if Weight <= 83.68:
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '<=', 83.68])
                                    if flow_duration <= 516.47:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 516.47])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if flow_duration > 516.47
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 516.47])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Weight > 83.68
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '>', 83.68])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if IAT > 78690612.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 78690612.0])
                                return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
            else:  # if rst_count > 0.5
                dict['rst_count'] = rst_count
                threshholds.append(['rst_count', '>', 0.5])
                if flow_duration <= 0.15:
                    dict['flow_duration'] = flow_duration
                    threshholds.append(['flow_duration', '<=', 0.15])
                    if Weight <= 126.97:
                        dict['Weight'] = Weight
                        threshholds.append(['Weight', '<=', 126.97])
                        if Min <= 60.5:
                            dict['Min'] = Min
                            threshholds.append(['Min', '<=', 60.5])
                            if psh_flag_number <= 0.5:
                                dict['psh_flag_number'] = psh_flag_number
                                threshholds.append(['psh_flag_number', '<=', 0.5])
                                if Max <= 80.8:
                                    dict['Max'] = Max
                                    threshholds.append(['Max', '<=', 80.8])
                                    if rst_flag_number <= 0.5:
                                        dict['rst_flag_number'] = rst_flag_number
                                        threshholds.append(['rst_flag_number', '<=', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.99622642,  0.00377358, 0.   ]], dict, threshholds
                                    else:  # if rst_flag_number > 0.5
                                        dict['rst_flag_number'] = rst_flag_number
                                        threshholds.append(['rst_flag_number', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if Max > 80.8
                                    dict['Max'] = Max
                                    threshholds.append(['Max', '>', 80.8])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                            else:  # if psh_flag_number > 0.5
                                dict['psh_flag_number'] = psh_flag_number
                                threshholds.append(['psh_flag_number', '>', 0.5])
                                return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if Min > 60.5
                            dict['Min'] = Min
                            threshholds.append(['Min', '>', 60.5])
                            if SSH <= 0.5:
                                dict['SSH'] = SSH
                                threshholds.append(['SSH', '<=', 0.5])
                                if Covariance <= 1.12:
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '<=', 1.12])
                                    if Header_Length <= 27577.7:
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '<=', 27577.7])
                                        return [[0.05555556, 0.16666667, 0.   , 0.   , 0.   , 0.77777778,  0.   , 0.   ]], dict, threshholds
                                    else:  # if Header_Length > 27577.7
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '>', 27577.7])
                                        return [[0.18181818, 0.   , 0.   , 0.   , 0.   , 0.   ,  0.45454545, 0.36363636]], dict, threshholds
                                else:  # if Covariance > 1.12
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '>', 1.12])
                                    if HTTP <= 0.5:
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '<=', 0.5])
                                        return [[0.   , 0.78947368, 0.   , 0.   , 0.   , 0.10526316,  0.   , 0.10526316]], dict, threshholds
                                    else:  # if HTTP > 0.5
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '>', 0.5])
                                        return [[0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5]], dict, threshholds
                            else:  # if SSH > 0.5
                                dict['SSH'] = SSH
                                threshholds.append(['SSH', '>', 0.5])
                                return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if Weight > 126.97
                        dict['Weight'] = Weight
                        threshholds.append(['Weight', '>', 126.97])
                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                else:  # if flow_duration > 0.15
                    dict['flow_duration'] = flow_duration
                    threshholds.append(['flow_duration', '>', 0.15])
                    if rst_count <= 287.0:
                        dict['rst_count'] = rst_count
                        threshholds.append(['rst_count', '<=', 287.0])
                        if Srate <= 6.95:
                            dict['Srate'] = Srate
                            threshholds.append(['Srate', '<=', 6.95])
                            if flow_duration <= 20.54:
                                dict['flow_duration'] = flow_duration
                                threshholds.append(['flow_duration', '<=', 20.54])
                                if Tot_sum <= 442.75:
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '<=', 442.75])
                                    if Tot_size <= 66.45:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 66.45])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if Tot_size > 66.45
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 66.45])
                                        return [[0.   , 0.8  , 0.   , 0.   , 0.   , 0.06666667,  0.13333333, 0.   ]], dict, threshholds
                                else:  # if Tot_sum > 442.75
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '>', 442.75])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if flow_duration > 20.54
                                dict['flow_duration'] = flow_duration
                                threshholds.append(['flow_duration', '>', 20.54])
                                if Min <= 67.7:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 67.7])
                                    if Srate <= 1.34:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 1.34])
                                        return [[0.05555556, 0.11111111, 0.   , 0.05555556, 0.   , 0.44444444,  0.11111111, 0.22222222]], dict, threshholds
                                    else:  # if Srate > 1.34
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 1.34])
                                        return [[0.12765957, 0.57446809, 0.   , 0.   , 0.   , 0.12765957,  0.12765957, 0.04255319]], dict, threshholds
                                else:  # if Min > 67.7
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 67.7])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                        else:  # if Srate > 6.95
                            dict['Srate'] = Srate
                            threshholds.append(['Srate', '>', 6.95])
                            if HTTPS <= 0.5:
                                dict['HTTPS'] = HTTPS
                                threshholds.append(['HTTPS', '<=', 0.5])
                                if IAT <= 41462536.02:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 41462536.02])
                                    if Magnitue <= 11.04:
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '<=', 11.04])
                                        return [[0.04347826, 0.04347826, 0.   , 0.   , 0.   , 0.83478261,  0.02608696, 0.05217391]], dict, threshholds
                                    else:  # if Magnitue > 11.04
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '>', 11.04])
                                        return [[0.03370787, 0.40449438, 0.   , 0.   , 0.   , 0.19101124,  0.07865169, 0.29213483]], dict, threshholds
                                else:  # if IAT > 41462536.02
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 41462536.02])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if HTTPS > 0.5
                                dict['HTTPS'] = HTTPS
                                threshholds.append(['HTTPS', '>', 0.5])
                                if Min <= 786.0:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 786.0])
                                    if rst_count <= 115.55:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 115.55])
                                        return [[0.10434783, 0.23478261, 0.   , 0.   , 0.   , 0.25217391,  0.16521739, 0.24347826]], dict, threshholds
                                    else:  # if rst_count > 115.55
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 115.55])
                                        return [[0.16483516, 0.10989011, 0.   , 0.   , 0.   , 0.13186813,  0.21978022, 0.37362637]], dict, threshholds
                                else:  # if Min > 786.0
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 786.0])
                                    if Rate <= 106.78:
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '<=', 106.78])
                                        return [[0.06896552, 0.10344828, 0.   , 0.   , 0.   , 0.06896552,  0.03448276, 0.72413793]], dict, threshholds
                                    else:  # if Rate > 106.78
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '>', 106.78])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                    else:  # if rst_count > 287.0
                        dict['rst_count'] = rst_count
                        threshholds.append(['rst_count', '>', 287.0])
                        if syn_count <= 0.7:
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '<=', 0.7])
                            if Rate <= 114.73:
                                dict['Rate'] = Rate
                                threshholds.append(['Rate', '<=', 114.73])
                                if urg_count <= 340.9:
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '<=', 340.9])
                                    if HTTP <= 0.5:
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '<=', 0.5])
                                        return [[0.75454545, 0.   , 0.   , 0.   , 0.   , 0.06363636,  0.09090909, 0.09090909]], dict, threshholds
                                    else:  # if HTTP > 0.5
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if urg_count > 340.9
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '>', 340.9])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if Rate > 114.73
                                dict['Rate'] = Rate
                                threshholds.append(['Rate', '>', 114.73])
                                if Min <= 1504.0:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 1504.0])
                                    if HTTP <= 0.5:
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '<=', 0.5])
                                        return [[0.03686636, 0.   , 0.   , 0.   , 0.   , 0.01843318,  0.94470046, 0.   ]], dict, threshholds
                                    else:  # if HTTP > 0.5
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '>', 0.5])
                                        return [[0.53846154, 0.   , 0.   , 0.   , 0.   , 0.   ,  0.46153846, 0.   ]], dict, threshholds
                                else:  # if Min > 1504.0
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 1504.0])
                                    if Duration <= 57.0:
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '<=', 57.0])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if Duration > 57.0
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '>', 57.0])
                                        return [[0.04, 0.  , 0.  , 0.  , 0.  , 0.04, 0.92, 0.  ]], dict, threshholds
                        else:  # if syn_count > 0.7
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '>', 0.7])
                            if IAT <= 0.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 0.0])
                                if Magnitue <= 53.6:
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '<=', 53.6])
                                    if HTTPS <= 0.5:
                                        dict['HTTPS'] = HTTPS
                                        threshholds.append(['HTTPS', '<=', 0.5])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if HTTPS > 0.5
                                        dict['HTTPS'] = HTTPS
                                        threshholds.append(['HTTPS', '>', 0.5])
                                        return [[0.32, 0.  , 0.  , 0.  , 0.  , 0.04, 0.48, 0.16]], dict, threshholds
                                else:  # if Magnitue > 53.6
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '>', 53.6])
                                    if Header_Length <= 427410.41:
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '<=', 427410.41])
                                        return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if Header_Length > 427410.41
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '>', 427410.41])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                            else:  # if IAT > 0.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 0.0])
                                if HTTPS <= 0.5:
                                    dict['HTTPS'] = HTTPS
                                    threshholds.append(['HTTPS', '<=', 0.5])
                                    if flow_duration <= 17.93:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 17.93])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if flow_duration > 17.93
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 17.93])
                                        return [[0.28571429, 0.14285714, 0.   , 0.   , 0.   , 0.28571429,  0.   , 0.28571429]], dict, threshholds
                                else:  # if HTTPS > 0.5
                                    dict['HTTPS'] = HTTPS
                                    threshholds.append(['HTTPS', '>', 0.5])
                                    if AVG <= 60.58:
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '<=', 60.58])
                                        return [[0.25 , 0.125, 0.   , 0.   , 0.   , 0.   , 0.125, 0.5  ]], dict, threshholds
                                    else:  # if AVG > 60.58
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '>', 60.58])
                                        return [[0.78172589, 0.05583756, 0.   , 0.   , 0.   , 0.08629442,  0.07614213, 0.   ]], dict, threshholds
        else:  # if IAT > 83033340.0
            dict['IAT'] = IAT
            threshholds.append(['IAT', '>', 83033340.0])
            if fin_count <= 0.0:
                dict['fin_count'] = fin_count
                threshholds.append(['fin_count', '<=', 0.0])
                if TCP <= 0.5:
                    dict['TCP'] = TCP
                    threshholds.append(['TCP', '<=', 0.5])
                    if ack_count <= 0.02:
                        dict['ack_count'] = ack_count
                        threshholds.append(['ack_count', '<=', 0.02])
                        if Tot_size <= 292.18:
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '<=', 292.18])
                            if Radius <= 5.41:
                                dict['Radius'] = Radius
                                threshholds.append(['Radius', '<=', 5.41])
                                if Tot_sum <= 700.96:
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '<=', 700.96])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Tot_sum > 700.96
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '>', 700.96])
                                    if Magnitue <= 10.68:
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '<=', 10.68])
                                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if Magnitue > 10.68
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '>', 10.68])
                                        return [[0.    , 0.    , 0.    , 0.    , 0.    , 0.0625, 0.9375, 0.    ]], dict, threshholds
                            else:  # if Radius > 5.41
                                dict['Radius'] = Radius
                                threshholds.append(['Radius', '>', 5.41])
                                if Magnitue <= 10.39:
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '<=', 10.39])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Magnitue > 10.39
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '>', 10.39])
                                    if Max <= 74.7:
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '<=', 74.7])
                                        return [[0.16666667, 0.41666667, 0.   , 0.   , 0.   , 0.   ,  0.25 , 0.16666667]], dict, threshholds
                                    else:  # if Max > 74.7
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '>', 74.7])
                                        return [[0.   , 0.   , 0.11111111, 0.   , 0.   , 0.   ,  0.88888889, 0.   ]], dict, threshholds
                        else:  # if Tot_size > 292.18
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '>', 292.18])
                            if Tot_size <= 902.01:
                                dict['Tot_size'] = Tot_size
                                threshholds.append(['Tot_size', '<=', 902.01])
                                return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                            else:  # if Tot_size > 902.01
                                dict['Tot_size'] = Tot_size
                                threshholds.append(['Tot_size', '>', 902.01])
                                if Duration <= 54.0:
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '<=', 54.0])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if Duration > 54.0
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '>', 54.0])
                                    if IAT <= 166849664.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166849664.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.77777778, 0.22222222]], dict, threshholds
                                    else:  # if IAT > 166849664.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166849664.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                    else:  # if ack_count > 0.02
                        dict['ack_count'] = ack_count
                        threshholds.append(['ack_count', '>', 0.02])
                        if Variance <= 0.64:
                            dict['Variance'] = Variance
                            threshholds.append(['Variance', '<=', 0.64])
                            if Min <= 290.12:
                                dict['Min'] = Min
                                threshholds.append(['Min', '<=', 290.12])
                                return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Min > 290.12
                                dict['Min'] = Min
                                threshholds.append(['Min', '>', 290.12])
                                return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                        else:  # if Variance > 0.64
                            dict['Variance'] = Variance
                            threshholds.append(['Variance', '>', 0.64])
                            return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                else:  # if TCP > 0.5
                    dict['TCP'] = TCP
                    threshholds.append(['TCP', '>', 0.5])
                    if Number <= 11.5:
                        dict['Number'] = Number
                        threshholds.append(['Number', '<=', 11.5])
                        if Number <= 7.5:
                            dict['Number'] = Number
                            threshholds.append(['Number', '<=', 7.5])
                            if HTTPS <= 0.5:
                                dict['HTTPS'] = HTTPS
                                threshholds.append(['HTTPS', '<=', 0.5])
                                return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                            else:  # if HTTPS > 0.5
                                dict['HTTPS'] = HTTPS
                                threshholds.append(['HTTPS', '>', 0.5])
                                return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                        else:  # if Number > 7.5
                            dict['Number'] = Number
                            threshholds.append(['Number', '>', 7.5])
                            return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if Number > 11.5
                        dict['Number'] = Number
                        threshholds.append(['Number', '>', 11.5])
                        if Std <= 0.38:
                            dict['Std'] = Std
                            threshholds.append(['Std', '<=', 0.38])
                            if ack_flag_number <= 0.5:
                                dict['ack_flag_number'] = ack_flag_number
                                threshholds.append(['ack_flag_number', '<=', 0.5])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if ack_flag_number > 0.5
                                dict['ack_flag_number'] = ack_flag_number
                                threshholds.append(['ack_flag_number', '>', 0.5])
                                if IAT <= 166729200.0:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 166729200.0])
                                    if IAT <= 166602848.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166602848.0])
                                        return [[0.59459459, 0.05405405, 0.   , 0.   , 0.   , 0.35135135,  0.   , 0.   ]], dict, threshholds
                                    else:  # if IAT > 166602848.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166602848.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if IAT > 166729200.0
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 166729200.0])
                                    if AVG <= 66.04:
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '<=', 66.04])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.11627907,  0.23255814, 0.65116279]], dict, threshholds
                                    else:  # if AVG > 66.04
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '>', 66.04])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.02020202, 0.97979798]], dict, threshholds
                        else:  # if Std > 0.38
                            dict['Std'] = Std
                            threshholds.append(['Std', '>', 0.38])
                            if Magnitue <= 11.12:
                                dict['Magnitue'] = Magnitue
                                threshholds.append(['Magnitue', '<=', 11.12])
                                if syn_flag_number <= 0.5:
                                    dict['syn_flag_number'] = syn_flag_number
                                    threshholds.append(['syn_flag_number', '<=', 0.5])
                                    if IAT <= 166605808.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166605808.0])
                                        return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if IAT > 166605808.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166605808.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if syn_flag_number > 0.5
                                    dict['syn_flag_number'] = syn_flag_number
                                    threshholds.append(['syn_flag_number', '>', 0.5])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if Magnitue > 11.12
                                dict['Magnitue'] = Magnitue
                                threshholds.append(['Magnitue', '>', 11.12])
                                if Duration <= 100.6:
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '<=', 100.6])
                                    if syn_count <= 0.1:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 0.1])
                                        return [[0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.9 , 0.05]], dict, threshholds
                                    else:  # if syn_count > 0.1
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 0.1])
                                        return [[0.   , 0.56097561, 0.   , 0.   , 0.   , 0.14634146,  0.09756098, 0.19512195]], dict, threshholds
                                else:  # if Duration > 100.6
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '>', 100.6])
                                    if Max <= 90.5:
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '<=', 90.5])
                                        return [[0.69565217, 0.   , 0.   , 0.   , 0.   , 0.04347826,  0.26086957, 0.   ]], dict, threshholds
                                    else:  # if Max > 90.5
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '>', 90.5])
                                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
            else:  # if fin_count > 0.0
                dict['fin_count'] = fin_count
                threshholds.append(['fin_count', '>', 0.0])
                if Weight <= 193.08:
                    dict['Weight'] = Weight
                    threshholds.append(['Weight', '<=', 193.08])
                    if fin_count <= 0.47:
                        dict['fin_count'] = fin_count
                        threshholds.append(['fin_count', '<=', 0.47])
                        if Max <= 64.31:
                            dict['Max'] = Max
                            threshholds.append(['Max', '<=', 64.31])
                            if AVG <= 56.22:
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '<=', 56.22])
                                return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if AVG > 56.22
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '>', 56.22])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                        else:  # if Max > 64.31
                            dict['Max'] = Max
                            threshholds.append(['Max', '>', 64.31])
                            if Max <= 364.0:
                                dict['Max'] = Max
                                threshholds.append(['Max', '<=', 364.0])
                                if fin_count <= 0.32:
                                    dict['fin_count'] = fin_count
                                    threshholds.append(['fin_count', '<=', 0.32])
                                    if IAT <= 83165664.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 83165664.0])
                                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if IAT > 83165664.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 83165664.0])
                                        return [[0. , 0. , 0.5, 0. , 0. , 0.5, 0. , 0. ]], dict, threshholds
                                else:  # if fin_count > 0.32
                                    dict['fin_count'] = fin_count
                                    threshholds.append(['fin_count', '>', 0.32])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if Max > 364.0
                                dict['Max'] = Max
                                threshholds.append(['Max', '>', 364.0])
                                return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                    else:  # if fin_count > 0.47
                        dict['fin_count'] = fin_count
                        threshholds.append(['fin_count', '>', 0.47])
                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                else:  # if Weight > 193.08
                    dict['Weight'] = Weight
                    threshholds.append(['Weight', '>', 193.08])
                    if Magnitue <= 11.14:
                        dict['Magnitue'] = Magnitue
                        threshholds.append(['Magnitue', '<=', 11.14])
                        if IAT <= 166850840.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 166850840.0])
                            if Max <= 54.2:
                                dict['Max'] = Max
                                threshholds.append(['Max', '<=', 54.2])
                                if AVG <= 53.88:
                                    dict['AVG'] = AVG
                                    threshholds.append(['AVG', '<=', 53.88])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if AVG > 53.88
                                    dict['AVG'] = AVG
                                    threshholds.append(['AVG', '>', 53.88])
                                    if Rate <= 7.23:
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '<=', 7.23])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.94117647,  0.05882353, 0.   ]], dict, threshholds
                                    else:  # if Rate > 7.23
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '>', 7.23])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if Max > 54.2
                                dict['Max'] = Max
                                threshholds.append(['Max', '>', 54.2])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                        else:  # if IAT > 166850840.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 166850840.0])
                            return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                    else:  # if Magnitue > 11.14
                        dict['Magnitue'] = Magnitue
                        threshholds.append(['Magnitue', '>', 11.14])
                        if Min <= 52.0:
                            dict['Min'] = Min
                            threshholds.append(['Min', '<=', 52.0])
                            return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if Min > 52.0
                            dict['Min'] = Min
                            threshholds.append(['Min', '>', 52.0])
                            return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
    else:  # if Std > 9.8
        dict['Std'] = Std
        threshholds.append(['Std', '>', 9.8])
        if IAT <= 166602848.0:
            dict['IAT'] = IAT
            threshholds.append(['IAT', '<=', 166602848.0])
            if Variance <= 0.39:
                dict['Variance'] = Variance
                threshholds.append(['Variance', '<=', 0.39])
                if IAT <= 83469472.0:
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '<=', 83469472.0])
                    if Header_Length <= 82721.44:
                        dict['Header_Length'] = Header_Length
                        threshholds.append(['Header_Length', '<=', 82721.44])
                        if Weight <= 140.4:
                            dict['Weight'] = Weight
                            threshholds.append(['Weight', '<=', 140.4])
                            if Protocol_Type <= 25.4:
                                dict['Protocol_Type'] = Protocol_Type
                                threshholds.append(['Protocol_Type', '<=', 25.4])
                                if HTTPS <= 0.5:
                                    dict['HTTPS'] = HTTPS
                                    threshholds.append(['HTTPS', '<=', 0.5])
                                    if SSH <= 0.5:
                                        dict['SSH'] = SSH
                                        threshholds.append(['SSH', '<=', 0.5])
                                        return [[0. , 0.1, 0. , 0. , 0. , 0.9, 0. , 0. ]], dict, threshholds
                                    else:  # if SSH > 0.5
                                        dict['SSH'] = SSH
                                        threshholds.append(['SSH', '>', 0.5])
                                        return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if HTTPS > 0.5
                                    dict['HTTPS'] = HTTPS
                                    threshholds.append(['HTTPS', '>', 0.5])
                                    if Rate <= 25.51:
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '<=', 25.51])
                                        return [[0.16666667, 0.16666667, 0.   , 0.   , 0.   , 0.   ,  0.   , 0.66666667]], dict, threshholds
                                    else:  # if Rate > 25.51
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '>', 25.51])
                                        return [[0.16, 0.24, 0.  , 0.  , 0.  , 0.08, 0.32, 0.2 ]], dict, threshholds
                            else:  # if Protocol_Type > 25.4
                                dict['Protocol_Type'] = Protocol_Type
                                threshholds.append(['Protocol_Type', '>', 25.4])
                                return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                        else:  # if Weight > 140.4
                            dict['Weight'] = Weight
                            threshholds.append(['Weight', '>', 140.4])
                            if Tot_sum <= 557.26:
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '<=', 557.26])
                                if Srate <= 107.41:
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '<=', 107.41])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Srate > 107.41
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '>', 107.41])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Tot_sum > 557.26
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '>', 557.26])
                                if Rate <= 3.37:
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '<=', 3.37])
                                    if IAT <= 83022948.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 83022948.0])
                                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if IAT > 83022948.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 83022948.0])
                                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Rate > 3.37
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '>', 3.37])
                                    if ICMP <= 0.5:
                                        dict['ICMP'] = ICMP
                                        threshholds.append(['ICMP', '<=', 0.5])
                                        return [[0.   , 0.   , 0.20532319, 0.79087452, 0.   , 0.00380228,  0.   , 0.   ]], dict, threshholds
                                    else:  # if ICMP > 0.5
                                        dict['ICMP'] = ICMP
                                        threshholds.append(['ICMP', '>', 0.5])
                                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if Header_Length > 82721.44
                        dict['Header_Length'] = Header_Length
                        threshholds.append(['Header_Length', '>', 82721.44])
                        if urg_count <= 188.85:
                            dict['urg_count'] = urg_count
                            threshholds.append(['urg_count', '<=', 188.85])
                            if urg_count <= 14.65:
                                dict['urg_count'] = urg_count
                                threshholds.append(['urg_count', '<=', 14.65])
                                if Header_Length <= 180764.0:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 180764.0])
                                    if ack_flag_number <= 0.5:
                                        dict['ack_flag_number'] = ack_flag_number
                                        threshholds.append(['ack_flag_number', '<=', 0.5])
                                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if ack_flag_number > 0.5
                                        dict['ack_flag_number'] = ack_flag_number
                                        threshholds.append(['ack_flag_number', '>', 0.5])
                                        return [[0.03125, 0.34375, 0.     , 0.     , 0.     , 0.     , 0.15625, 0.46875]], dict, threshholds
                                else:  # if Header_Length > 180764.0
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 180764.0])
                                    if Radius <= 48.07:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 48.07])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.44444444, 0.   ,  0.44444444, 0.11111111]], dict, threshholds
                                    else:  # if Radius > 48.07
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 48.07])
                                        return [[0.05555556, 0.   , 0.   , 0.02777778, 0.   , 0.   ,  0.66666667, 0.25 ]], dict, threshholds
                            else:  # if urg_count > 14.65
                                dict['urg_count'] = urg_count
                                threshholds.append(['urg_count', '>', 14.65])
                                if Tot_size <= 2347.6:
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '<=', 2347.6])
                                    if Header_Length <= 1000736.09:
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '<=', 1000736.09])
                                        return [[0.31034483, 0.10344828, 0.   , 0.   , 0.   , 0.20689655,  0.18965517, 0.18965517]], dict, threshholds
                                    else:  # if Header_Length > 1000736.09
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '>', 1000736.09])
                                        return [[0.87234043, 0.   , 0.   , 0.   , 0.   , 0.   ,  0.08510638, 0.04255319]], dict, threshholds
                                else:  # if Tot_size > 2347.6
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '>', 2347.6])
                                    if Covariance <= 915674.0:
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '<=', 915674.0])
                                        return [[0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0.8]], dict, threshholds
                                    else:  # if Covariance > 915674.0
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '>', 915674.0])
                                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if urg_count > 188.85
                            dict['urg_count'] = urg_count
                            threshholds.append(['urg_count', '>', 188.85])
                            if HTTP <= 0.5:
                                dict['HTTP'] = HTTP
                                threshholds.append(['HTTP', '<=', 0.5])
                                if rst_count <= 6170.85:
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '<=', 6170.85])
                                    if Tot_sum <= 435.1:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 435.1])
                                        return [[0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0. ]], dict, threshholds
                                    else:  # if Tot_sum > 435.1
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 435.1])
                                        return [[0.025, 0.   , 0.   , 0.   , 0.   , 0.   , 0.925, 0.05 ]], dict, threshholds
                                else:  # if rst_count > 6170.85
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '>', 6170.85])
                                    if psh_flag_number <= 0.5:
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '<=', 0.5])
                                        return [[0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0. ]], dict, threshholds
                                    else:  # if psh_flag_number > 0.5
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if HTTP > 0.5
                                dict['HTTP'] = HTTP
                                threshholds.append(['HTTP', '>', 0.5])
                                return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                else:  # if IAT > 83469472.0
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '>', 83469472.0])
                    if IAT <= 133481500.0:
                        dict['IAT'] = IAT
                        threshholds.append(['IAT', '<=', 133481500.0])
                        return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                    else:  # if IAT > 133481500.0
                        dict['IAT'] = IAT
                        threshholds.append(['IAT', '>', 133481500.0])
                        if Tot_size <= 98.2:
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '<=', 98.2])
                            return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if Tot_size > 98.2
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '>', 98.2])
                            if syn_count <= 1.7:
                                dict['syn_count'] = syn_count
                                threshholds.append(['syn_count', '<=', 1.7])
                                if Rate <= 74.44:
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '<=', 74.44])
                                    return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Rate > 74.44
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '>', 74.44])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if syn_count > 1.7
                                dict['syn_count'] = syn_count
                                threshholds.append(['syn_count', '>', 1.7])
                                if Srate <= 73.62:
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '<=', 73.62])
                                    return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Srate > 73.62
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '>', 73.62])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
            else:  # if Variance > 0.39
                dict['Variance'] = Variance
                threshholds.append(['Variance', '>', 0.39])
                if HTTPS <= 0.5:
                    dict['HTTPS'] = HTTPS
                    threshholds.append(['HTTPS', '<=', 0.5])
                    if Number <= 12.75:
                        dict['Number'] = Number
                        threshholds.append(['Number', '<=', 12.75])
                        if Max <= 466.88:
                            dict['Max'] = Max
                            threshholds.append(['Max', '<=', 466.88])
                            if Number <= 7.5:
                                dict['Number'] = Number
                                threshholds.append(['Number', '<=', 7.5])
                                if Header_Length <= 150021.5:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 150021.5])
                                    if TCP <= 0.5:
                                        dict['TCP'] = TCP
                                        threshholds.append(['TCP', '<=', 0.5])
                                        return [[0.10306083, 0.25261527, 0.   , 0.   , 0.   , 0.1483921 ,  0.11390934, 0.38202247]], dict, threshholds
                                    else:  # if TCP > 0.5
                                        dict['TCP'] = TCP
                                        threshholds.append(['TCP', '>', 0.5])
                                        return [[0.08537693, 0.35603996, 0.   , 0.   , 0.   , 0.25158946,  0.06176203, 0.24523161]], dict, threshholds
                                else:  # if Header_Length > 150021.5
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 150021.5])
                                    if Protocol_Type <= 12.65:
                                        dict['Protocol_Type'] = Protocol_Type
                                        threshholds.append(['Protocol_Type', '<=', 12.65])
                                        return [[0.59776536, 0.02234637, 0.   , 0.   , 0.   , 0.08938547,  0.20670391, 0.08379888]], dict, threshholds
                                    else:  # if Protocol_Type > 12.65
                                        dict['Protocol_Type'] = Protocol_Type
                                        threshholds.append(['Protocol_Type', '>', 12.65])
                                        return [[0.05555556, 0.   , 0.   , 0.   , 0.   , 0.00617284,  0.92592593, 0.01234568]], dict, threshholds
                            else:  # if Number > 7.5
                                dict['Number'] = Number
                                threshholds.append(['Number', '>', 7.5])
                                if HTTP <= 0.5:
                                    dict['HTTP'] = HTTP
                                    threshholds.append(['HTTP', '<=', 0.5])
                                    if rst_count <= 1.32:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 1.32])
                                        return [[0.   , 0.   , 0.25 , 0.58333333, 0.05555556, 0.11111111,  0.   , 0.   ]], dict, threshholds
                                    else:  # if rst_count > 1.32
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 1.32])
                                        return [[0.   , 0.   , 0.00797872, 0.03191489, 0.02659574, 0.93351064,  0.   , 0.   ]], dict, threshholds
                                else:  # if HTTP > 0.5
                                    dict['HTTP'] = HTTP
                                    threshholds.append(['HTTP', '>', 0.5])
                                    if flow_duration <= 0.59:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 0.59])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if flow_duration > 0.59
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 0.59])
                                        return [[0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ]], dict, threshholds
                        else:  # if Max > 466.88
                            dict['Max'] = Max
                            threshholds.append(['Max', '>', 466.88])
                            if Weight <= 79.94:
                                dict['Weight'] = Weight
                                threshholds.append(['Weight', '<=', 79.94])
                                if UDP <= 0.5:
                                    dict['UDP'] = UDP
                                    threshholds.append(['UDP', '<=', 0.5])
                                    if Radius <= 753.63:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 753.63])
                                        return [[0.11621622, 0.38513514, 0.00135135, 0.   , 0.   , 0.07162162,  0.13108108, 0.29459459]], dict, threshholds
                                    else:  # if Radius > 753.63
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 753.63])
                                        return [[0.32535885, 0.03110048, 0.   , 0.   , 0.   , 0.11722488,  0.38995215, 0.13636364]], dict, threshholds
                                else:  # if UDP > 0.5
                                    dict['UDP'] = UDP
                                    threshholds.append(['UDP', '>', 0.5])
                                    if Srate <= 29.92:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 29.92])
                                        return [[0.14516129, 0.09677419, 0.   , 0.   , 0.   , 0.22580645,  0.37096774, 0.16129032]], dict, threshholds
                                    else:  # if Srate > 29.92
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 29.92])
                                        return [[0.0195122 , 0.00731707, 0.   , 0.   , 0.   , 0.01219512,  0.9195122 , 0.04146341]], dict, threshholds
                            else:  # if Weight > 79.94
                                dict['Weight'] = Weight
                                threshholds.append(['Weight', '>', 79.94])
                                if Variance <= 0.83:
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '<=', 0.83])
                                    if rst_count <= 5.44:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 5.44])
                                        return [[0.   , 0.   , 0.01421801, 0.02843602, 0.95260664, 0.00473934,  0.   , 0.   ]], dict, threshholds
                                    else:  # if rst_count > 5.44
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 5.44])
                                        return [[0.   , 0.   , 0.02631579, 0.60526316, 0.23684211, 0.13157895,  0.   , 0.   ]], dict, threshholds
                                else:  # if Variance > 0.83
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '>', 0.83])
                                    if Tot_size <= 478.82:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 478.82])
                                        return [[0.   , 0.   , 0.0199005 , 0.04477612, 0.06467662, 0.86567164,  0.   , 0.00497512]], dict, threshholds
                                    else:  # if Tot_size > 478.82
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 478.82])
                                        return [[0.   , 0.   , 0.74770642, 0.0733945 , 0.01834862, 0.16055046,  0.   , 0.   ]], dict, threshholds
                    else:  # if Number > 12.75
                        dict['Number'] = Number
                        threshholds.append(['Number', '>', 12.75])
                        if IAT <= 166551400.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 166551400.0])
                            if IAT <= 166499176.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 166499176.0])
                                if Tot_size <= 85.75:
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '<=', 85.75])
                                    if syn_count <= 0.45:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 0.45])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.55371901,  0.   , 0.44628099]], dict, threshholds
                                    else:  # if syn_count > 0.45
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 0.45])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.86982249,  0.   , 0.13017751]], dict, threshholds
                                else:  # if Tot_size > 85.75
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '>', 85.75])
                                    if Tot_sum <= 3884.2:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 3884.2])
                                        return [[0.001221  , 0.001221  , 0.   , 0.   , 0.   , 0.51526252,  0.   , 0.48229548]], dict, threshholds
                                    else:  # if Tot_sum > 3884.2
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 3884.2])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.84, 0.  , 0.16]], dict, threshholds
                            else:  # if IAT > 166499176.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 166499176.0])
                                return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if IAT > 166551400.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 166551400.0])
                            return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                else:  # if HTTPS > 0.5
                    dict['HTTPS'] = HTTPS
                    threshholds.append(['HTTPS', '>', 0.5])
                    if Rate <= 292.85:
                        dict['Rate'] = Rate
                        threshholds.append(['Rate', '<=', 292.85])
                        if rst_count <= 284.45:
                            dict['rst_count'] = rst_count
                            threshholds.append(['rst_count', '<=', 284.45])
                            if Max <= 873.25:
                                dict['Max'] = Max
                                threshholds.append(['Max', '<=', 873.25])
                                if ack_count <= 0.13:
                                    dict['ack_count'] = ack_count
                                    threshholds.append(['ack_count', '<=', 0.13])
                                    if fin_count <= 0.08:
                                        dict['fin_count'] = fin_count
                                        threshholds.append(['fin_count', '<=', 0.08])
                                        return [[0.16735113, 0.30595483, 0.   , 0.00308008, 0.   , 0.137577  ,  0.11601643, 0.27002053]], dict, threshholds
                                    else:  # if fin_count > 0.08
                                        dict['fin_count'] = fin_count
                                        threshholds.append(['fin_count', '>', 0.08])
                                        return [[0.03636364, 0.54545455, 0.   , 0.   , 0.   , 0.18181818,  0.14545455, 0.09090909]], dict, threshholds
                                else:  # if ack_count > 0.13
                                    dict['ack_count'] = ack_count
                                    threshholds.append(['ack_count', '>', 0.13])
                                    if Max <= 140.5:
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '<=', 140.5])
                                        return [[0.1 , 0.36, 0.  , 0.  , 0.  , 0.26, 0.16, 0.12]], dict, threshholds
                                    else:  # if Max > 140.5
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '>', 140.5])
                                        return [[0.16993464, 0.15686275, 0.00217865, 0.   , 0.   , 0.30501089,  0.05010893, 0.31590414]], dict, threshholds
                            else:  # if Max > 873.25
                                dict['Max'] = Max
                                threshholds.append(['Max', '>', 873.25])
                                if Variance <= 0.97:
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '<=', 0.97])
                                    if Header_Length <= 156572.95:
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '<=', 156572.95])
                                        return [[0.13608247, 0.36804124, 0.   , 0.00515464, 0.00515464, 0.13092784,  0.12989691, 0.22474227]], dict, threshholds
                                    else:  # if Header_Length > 156572.95
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '>', 156572.95])
                                        return [[0.49717514, 0.06779661, 0.   , 0.   , 0.00564972, 0.11864407,  0.23728814, 0.07344633]], dict, threshholds
                                else:  # if Variance > 0.97
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '>', 0.97])
                                    if syn_count <= 0.65:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 0.65])
                                        return [[0.64754098, 0.17213115, 0.   , 0.   , 0.   , 0.13934426,  0.   , 0.04098361]], dict, threshholds
                                    else:  # if syn_count > 0.65
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 0.65])
                                        return [[0.25775656, 0.55250597, 0.   , 0.   , 0.   , 0.13365155,  0.   , 0.05608592]], dict, threshholds
                        else:  # if rst_count > 284.45
                            dict['rst_count'] = rst_count
                            threshholds.append(['rst_count', '>', 284.45])
                            if Covariance <= 19281.41:
                                dict['Covariance'] = Covariance
                                threshholds.append(['Covariance', '<=', 19281.41])
                                if Header_Length <= 220049.1:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 220049.1])
                                    if IAT <= 166551376.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166551376.0])
                                        return [[0.33647059, 0.28470588, 0.   , 0.   , 0.   , 0.24941176,  0.04 , 0.08941176]], dict, threshholds
                                    else:  # if IAT > 166551376.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166551376.0])
                                        return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Header_Length > 220049.1
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 220049.1])
                                    if IAT <= 166473832.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166473832.0])
                                        return [[0.72875817, 0.05228758, 0.   , 0.   , 0.   , 0.12418301,  0.09477124, 0.   ]], dict, threshholds
                                    else:  # if IAT > 166473832.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166473832.0])
                                        return [[0.98461538, 0.01538462, 0.   , 0.   , 0.   , 0.   ,  0.   , 0.   ]], dict, threshholds
                            else:  # if Covariance > 19281.41
                                dict['Covariance'] = Covariance
                                threshholds.append(['Covariance', '>', 19281.41])
                                if Min <= 67.95:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 67.95])
                                    if Weight <= 193.08:
                                        dict['Weight'] = Weight
                                        threshholds.append(['Weight', '<=', 193.08])
                                        return [[0.62383613, 0.0707635 , 0.   , 0.   , 0.   , 0.11731844,  0.17318436, 0.01489758]], dict, threshholds
                                    else:  # if Weight > 193.08
                                        dict['Weight'] = Weight
                                        threshholds.append(['Weight', '>', 193.08])
                                        return [[0.81155433, 0.08046768, 0.   , 0.   , 0.   , 0.10660248,  0.   , 0.00137552]], dict, threshholds
                                else:  # if Min > 67.95
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 67.95])
                                    if psh_flag_number <= 0.5:
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '<=', 0.5])
                                        return [[0.70740741, 0.06111111, 0.   , 0.00185185, 0.   , 0.09444444,  0.1287037 , 0.00648148]], dict, threshholds
                                    else:  # if psh_flag_number > 0.5
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '>', 0.5])
                                        return [[0.17460317, 0.03174603, 0.   , 0.   , 0.   , 0.06349206,  0.71428571, 0.01587302]], dict, threshholds
                    else:  # if Rate > 292.85
                        dict['Rate'] = Rate
                        threshholds.append(['Rate', '>', 292.85])
                        if syn_count <= 0.45:
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '<=', 0.45])
                            if AVG <= 1382.54:
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '<=', 1382.54])
                                if Variance <= 0.95:
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '<=', 0.95])
                                    if IAT <= 0.01:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 0.01])
                                        return [[0.04580153, 0.00763359, 0.   , 0.   , 0.   , 0.01526718,  0.90839695, 0.02290076]], dict, threshholds
                                    else:  # if IAT > 0.01
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 0.01])
                                        return [[0.35714286, 0.07142857, 0.   , 0.   , 0.14285714, 0.03571429,  0.25 , 0.14285714]], dict, threshholds
                                else:  # if Variance > 0.95
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '>', 0.95])
                                    if syn_count <= 0.25:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 0.25])
                                        return [[0.68181818, 0.09090909, 0.   , 0.   , 0.   , 0.22727273,  0.   , 0.   ]], dict, threshholds
                                    else:  # if syn_count > 0.25
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 0.25])
                                        return [[0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5]], dict, threshholds
                            else:  # if AVG > 1382.54
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '>', 1382.54])
                                if IAT <= 83219648.01:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 83219648.01])
                                    if urg_count <= 86.1:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 86.1])
                                        return [[0.03809524, 0.   , 0.   , 0.   , 0.   , 0.01904762,  0.85714286, 0.08571429]], dict, threshholds
                                    else:  # if urg_count > 86.1
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 86.1])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if IAT > 83219648.01
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 83219648.01])
                                    if rst_count <= 178.75:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 178.75])
                                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if rst_count > 178.75
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 178.75])
                                        return [[0.75, 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  ]], dict, threshholds
                        else:  # if syn_count > 0.45
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '>', 0.45])
                            if Max <= 3093.8:
                                dict['Max'] = Max
                                threshholds.append(['Max', '<=', 3093.8])
                                if Weight <= 90.03:
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '<=', 90.03])
                                    if rst_count <= 1266.6:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 1266.6])
                                        return [[0.18333333, 0.13333333, 0.   , 0.   , 0.   , 0.1  ,  0.26666667, 0.31666667]], dict, threshholds
                                    else:  # if rst_count > 1266.6
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 1266.6])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.75]], dict, threshholds
                                else:  # if Weight > 90.03
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '>', 90.03])
                                    if rst_count <= 89.1:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 89.1])
                                        return [[0.   , 0.33333333, 0.   , 0.   , 0.11111111, 0.44444444,  0.   , 0.11111111]], dict, threshholds
                                    else:  # if rst_count > 89.1
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 89.1])
                                        return [[0.61904762, 0.14285714, 0.04761905, 0.   , 0.   , 0.19047619,  0.   , 0.   ]], dict, threshholds
                            else:  # if Max > 3093.8
                                dict['Max'] = Max
                                threshholds.append(['Max', '>', 3093.8])
                                if Covariance <= 2317899.75:
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '<=', 2317899.75])
                                    if Number <= 9.5:
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '<=', 9.5])
                                        return [[0.   , 0.21428571, 0.   , 0.   , 0.   , 0.07142857,  0.67857143, 0.03571429]], dict, threshholds
                                    else:  # if Number > 9.5
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '>', 9.5])
                                        return [[0.125, 0.75 , 0.   , 0.   , 0.   , 0.125, 0.   , 0.   ]], dict, threshholds
                                else:  # if Covariance > 2317899.75
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '>', 2317899.75])
                                    if Duration <= 62.75:
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '<=', 62.75])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.75, 0.  ]], dict, threshholds
                                    else:  # if Duration > 62.75
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '>', 62.75])
                                        return [[0.5, 0.1, 0. , 0. , 0. , 0. , 0.2, 0.2]], dict, threshholds
        else:  # if IAT > 166602848.0
            dict['IAT'] = IAT
            threshholds.append(['IAT', '>', 166602848.0])
            if Radius <= 278.51:
                dict['Radius'] = Radius
                threshholds.append(['Radius', '<=', 278.51])
                if IAT <= 167246344.0:
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '<=', 167246344.0])
                    if syn_count <= 0.05:
                        dict['syn_count'] = syn_count
                        threshholds.append(['syn_count', '<=', 0.05])
                        if Duration <= 84.4:
                            dict['Duration'] = Duration
                            threshholds.append(['Duration', '<=', 84.4])
                            if rst_count <= 4.3:
                                dict['rst_count'] = rst_count
                                threshholds.append(['rst_count', '<=', 4.3])
                                if Covariance <= 6269.43:
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '<=', 6269.43])
                                    if UDP <= 0.5:
                                        dict['UDP'] = UDP
                                        threshholds.append(['UDP', '<=', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.33333333,  0.58333333, 0.08333333]], dict, threshholds
                                    else:  # if UDP > 0.5
                                        dict['UDP'] = UDP
                                        threshholds.append(['UDP', '>', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.05454545,  0.90909091, 0.03636364]], dict, threshholds
                                else:  # if Covariance > 6269.43
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '>', 6269.43])
                                    if Rate <= 28.62:
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '<=', 28.62])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.89473684, 0.10526316]], dict, threshholds
                                    else:  # if Rate > 28.62
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '>', 28.62])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if rst_count > 4.3
                                dict['rst_count'] = rst_count
                                threshholds.append(['rst_count', '>', 4.3])
                                if Tot_sum <= 1616.65:
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '<=', 1616.65])
                                    if HTTP <= 0.5:
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '<=', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.08139535,  0.84883721, 0.06976744]], dict, threshholds
                                    else:  # if HTTP > 0.5
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Tot_sum > 1616.65
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '>', 1616.65])
                                    if urg_count <= 64.35:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 64.35])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.15384615,  0.5  , 0.34615385]], dict, threshholds
                                    else:  # if urg_count > 64.35
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 64.35])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.125, 0.75 , 0.125]], dict, threshholds
                        else:  # if Duration > 84.4
                            dict['Duration'] = Duration
                            threshholds.append(['Duration', '>', 84.4])
                            if Header_Length <= 221395.0:
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '<=', 221395.0])
                                if rst_count <= 240.9:
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '<=', 240.9])
                                    if AVG <= 98.91:
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '<=', 98.91])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.2295082,  0.6557377, 0.1147541]], dict, threshholds
                                    else:  # if AVG > 98.91
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '>', 98.91])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.26180258,  0.41630901, 0.32188841]], dict, threshholds
                                else:  # if rst_count > 240.9
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '>', 240.9])
                                    if Covariance <= 4959.5:
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '<=', 4959.5])
                                        return [[0.   , 0.02325581, 0.   , 0.   , 0.   , 0.76744186,  0.20930233, 0.   ]], dict, threshholds
                                    else:  # if Covariance > 4959.5
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '>', 4959.5])
                                        return [[0. , 0. , 0. , 0. , 0. , 0.3, 0.6, 0.1]], dict, threshholds
                            else:  # if Header_Length > 221395.0
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '>', 221395.0])
                                if Radius <= 56.66:
                                    dict['Radius'] = Radius
                                    threshholds.append(['Radius', '<=', 56.66])
                                    if Srate <= 34.16:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 34.16])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if Srate > 34.16
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 34.16])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Radius > 56.66
                                    dict['Radius'] = Radius
                                    threshholds.append(['Radius', '>', 56.66])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                    else:  # if syn_count > 0.05
                        dict['syn_count'] = syn_count
                        threshholds.append(['syn_count', '>', 0.05])
                        if fin_count <= 0.05:
                            dict['fin_count'] = fin_count
                            threshholds.append(['fin_count', '<=', 0.05])
                            if Covariance <= 18379.3:
                                dict['Covariance'] = Covariance
                                threshholds.append(['Covariance', '<=', 18379.3])
                                if syn_count <= 0.15:
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '<=', 0.15])
                                    if flow_duration <= 72.36:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 72.36])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.47058824,  0.47058824, 0.05882353]], dict, threshholds
                                    else:  # if flow_duration > 72.36
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 72.36])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.85714286,  0.11428571, 0.02857143]], dict, threshholds
                                else:  # if syn_count > 0.15
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '>', 0.15])
                                    if Radius <= 69.8:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 69.8])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.24299065,  0.58878505, 0.1682243 ]], dict, threshholds
                                    else:  # if Radius > 69.8
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 69.8])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.42618384,  0.35933148, 0.21448468]], dict, threshholds
                            else:  # if Covariance > 18379.3
                                dict['Covariance'] = Covariance
                                threshholds.append(['Covariance', '>', 18379.3])
                                if IAT <= 166846232.0:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 166846232.0])
                                    if Duration <= 160.4:
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '<=', 160.4])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.21495327, 0.78504673]], dict, threshholds
                                    else:  # if Duration > 160.4
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '>', 160.4])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.83333333, 0.16666667]], dict, threshholds
                                else:  # if IAT > 166846232.0
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 166846232.0])
                                    if urg_count <= 36.65:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 36.65])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.18518519,  0.81481481, 0.   ]], dict, threshholds
                                    else:  # if urg_count > 36.65
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 36.65])
                                        return [[0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ]], dict, threshholds
                        else:  # if fin_count > 0.05
                            dict['fin_count'] = fin_count
                            threshholds.append(['fin_count', '>', 0.05])
                            if Tot_sum <= 1398.8:
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '<=', 1398.8])
                                if syn_count <= 1.75:
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '<=', 1.75])
                                    if Protocol_Type <= 10.7:
                                        dict['Protocol_Type'] = Protocol_Type
                                        threshholds.append(['Protocol_Type', '<=', 10.7])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.98086124,  0.01913876, 0.   ]], dict, threshholds
                                    else:  # if Protocol_Type > 10.7
                                        dict['Protocol_Type'] = Protocol_Type
                                        threshholds.append(['Protocol_Type', '>', 10.7])
                                        return [[0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ]], dict, threshholds
                                else:  # if syn_count > 1.75
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '>', 1.75])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if Tot_sum > 1398.8
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '>', 1398.8])
                                if Header_Length <= 23286.3:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 23286.3])
                                    if Radius <= 81.55:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 81.55])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.22222222,  0.77777778, 0.   ]], dict, threshholds
                                    else:  # if Radius > 81.55
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 81.55])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.73469388,  0.18367347, 0.08163265]], dict, threshholds
                                else:  # if Header_Length > 23286.3
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 23286.3])
                                    if IAT <= 166729120.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166729120.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if IAT > 166729120.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166729120.0])
                                        return [[0. , 0. , 0. , 0. , 0. , 0.2, 0.5, 0.3]], dict, threshholds
                else:  # if IAT > 167246344.0
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '>', 167246344.0])
                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
            else:  # if Radius > 278.51
                dict['Radius'] = Radius
                threshholds.append(['Radius', '>', 278.51])
                if IAT <= 167246344.0:
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '<=', 167246344.0])
                    if Srate <= 22.49:
                        dict['Srate'] = Srate
                        threshholds.append(['Srate', '<=', 22.49])
                        if AVG <= 203.92:
                            dict['AVG'] = AVG
                            threshholds.append(['AVG', '<=', 203.92])
                            if IAT <= 166850728.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 166850728.0])
                                if Duration <= 116.55:
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '<=', 116.55])
                                    if psh_flag_number <= 0.5:
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '<=', 0.5])
                                        return [[0.    , 0.    , 0.    , 0.    , 0.    , 0.25  , 0.5625, 0.1875]], dict, threshholds
                                    else:  # if psh_flag_number > 0.5
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Duration > 116.55
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '>', 116.55])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if IAT > 166850728.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 166850728.0])
                                return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                        else:  # if AVG > 203.92
                            dict['AVG'] = AVG
                            threshholds.append(['AVG', '>', 203.92])
                            if Header_Length <= 7715.2:
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '<=', 7715.2])
                                if rst_count <= 39.4:
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '<=', 39.4])
                                    if Covariance <= 86125.98:
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '<=', 86125.98])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.83333333, 0.16666667]], dict, threshholds
                                    else:  # if Covariance > 86125.98
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '>', 86125.98])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.51219512,  0.12195122, 0.36585366]], dict, threshholds
                                else:  # if rst_count > 39.4
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '>', 39.4])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                            else:  # if Header_Length > 7715.2
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '>', 7715.2])
                                if urg_count <= 12.1:
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '<=', 12.1])
                                    if Max <= 3076.5:
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '<=', 3076.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.04166667,  0.91666667, 0.04166667]], dict, threshholds
                                    else:  # if Max > 3076.5
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '>', 3076.5])
                                        return [[0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5]], dict, threshholds
                                else:  # if urg_count > 12.1
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '>', 12.1])
                                    if Covariance <= 1051078.34:
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '<=', 1051078.34])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.09574468,  0.4893617 , 0.41489362]], dict, threshholds
                                    else:  # if Covariance > 1051078.34
                                        dict['Covariance'] = Covariance
                                        threshholds.append(['Covariance', '>', 1051078.34])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.19047619,  0.80952381, 0.   ]], dict, threshholds
                    else:  # if Srate > 22.49
                        dict['Srate'] = Srate
                        threshholds.append(['Srate', '>', 22.49])
                        if syn_count <= 1.55:
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '<=', 1.55])
                            if flow_duration <= 0.2:
                                dict['flow_duration'] = flow_duration
                                threshholds.append(['flow_duration', '<=', 0.2])
                                if Rate <= 1169.08:
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '<=', 1169.08])
                                    if fin_count <= 0.35:
                                        dict['fin_count'] = fin_count
                                        threshholds.append(['fin_count', '<=', 0.35])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.07317073,  0.53658537, 0.3902439 ]], dict, threshholds
                                    else:  # if fin_count > 0.35
                                        dict['fin_count'] = fin_count
                                        threshholds.append(['fin_count', '>', 0.35])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if Rate > 1169.08
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '>', 1169.08])
                                    if Tot_sum <= 37934.4:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 37934.4])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if Tot_sum > 37934.4
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 37934.4])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25]], dict, threshholds
                            else:  # if flow_duration > 0.2
                                dict['flow_duration'] = flow_duration
                                threshholds.append(['flow_duration', '>', 0.2])
                                if Covariance <= 81486.11:
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '<=', 81486.11])
                                    if TCP <= 0.5:
                                        dict['TCP'] = TCP
                                        threshholds.append(['TCP', '<=', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.91176471, 0.08823529]], dict, threshholds
                                    else:  # if TCP > 0.5
                                        dict['TCP'] = TCP
                                        threshholds.append(['TCP', '>', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.22222222,  0.38888889, 0.38888889]], dict, threshholds
                                else:  # if Covariance > 81486.11
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '>', 81486.11])
                                    if Tot_size <= 350.8:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 350.8])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.04057971,  0.90724638, 0.05217391]], dict, threshholds
                                    else:  # if Tot_size > 350.8
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 350.8])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.00622222,  0.968     , 0.02577778]], dict, threshholds
                        else:  # if syn_count > 1.55
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '>', 1.55])
                            if flow_duration <= 3.07:
                                dict['flow_duration'] = flow_duration
                                threshholds.append(['flow_duration', '<=', 3.07])
                                if Duration <= 246.5:
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '<=', 246.5])
                                    if IAT <= 166729192.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166729192.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if IAT > 166729192.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166729192.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.12565445,  0.20418848, 0.67015707]], dict, threshholds
                                else:  # if Duration > 246.5
                                    dict['Duration'] = Duration
                                    threshholds.append(['Duration', '>', 246.5])
                                    if IAT <= 166729152.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166729152.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if IAT > 166729152.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166729152.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.875, 0.125, 0.   ]], dict, threshholds
                            else:  # if flow_duration > 3.07
                                dict['flow_duration'] = flow_duration
                                threshholds.append(['flow_duration', '>', 3.07])
                                if flow_duration <= 3.52:
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '<=', 3.52])
                                    if Rate <= 372.16:
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '<=', 372.16])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if Rate > 372.16
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '>', 372.16])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.66666667, 0.33333333]], dict, threshholds
                                else:  # if flow_duration > 3.52
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '>', 3.52])
                                    if rst_count <= 732.1:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 732.1])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.11538462,  0.71153846, 0.17307692]], dict, threshholds
                                    else:  # if rst_count > 732.1
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 732.1])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.0754717 ,  0.52830189, 0.39622642]], dict, threshholds
                else:  # if IAT > 167246344.0
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '>', 167246344.0])
                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds

In [11]:
def predict_2(flow_duration, Header_Length, Protocol_Type, Duration, Rate, Srate, Drate, fin_flag_number, syn_flag_number, rst_flag_number, psh_flag_number, ack_flag_number, ece_flag_number, cwr_flag_number, ack_count, syn_count, fin_count, urg_count, rst_count, HTTP, HTTPS, DNS, Telnet, SMTP, SSH, IRC, TCP, UDP, DHCP, ARP, ICMP, IPv, LLC, Tot_sum, Min, Max, AVG, Std, Tot_size, IAT, Number, Magnitue, Radius, Covariance, Variance, Weight):
    dict = {}
    threshholds = []
    if Variance <= 0.39:
        dict['Variance'] = Variance
        threshholds.append(['Variance', '<=', 0.39])
        if Tot_sum <= 3517.28:
            dict['Tot_sum'] = Tot_sum
            threshholds.append(['Tot_sum', '<=', 3517.28])
            if Header_Length <= 79.93:
                dict['Header_Length'] = Header_Length
                threshholds.append(['Header_Length', '<=', 79.93])
                if TCP <= 0.5:
                    dict['TCP'] = TCP
                    threshholds.append(['TCP', '<=', 0.5])
                    if Protocol_Type <= 1.5:
                        dict['Protocol_Type'] = Protocol_Type
                        threshholds.append(['Protocol_Type', '<=', 1.5])
                        if LLC <= 0.5:
                            dict['LLC'] = LLC
                            threshholds.append(['LLC', '<=', 0.5])
                            if Srate <= 0.23:
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '<=', 0.23])
                                if Rate <= 0.14:
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '<=', 0.14])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Rate > 0.14
                                    dict['Rate'] = Rate
                                    threshholds.append(['Rate', '>', 0.14])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if Srate > 0.23
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '>', 0.23])
                                return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                        else:  # if LLC > 0.5
                            dict['LLC'] = LLC
                            threshholds.append(['LLC', '>', 0.5])
                            if Radius <= 1.72:
                                dict['Radius'] = Radius
                                threshholds.append(['Radius', '<=', 1.72])
                                return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Radius > 1.72
                                dict['Radius'] = Radius
                                threshholds.append(['Radius', '>', 1.72])
                                if Min <= 54.76:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 54.76])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Min > 54.76
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 54.76])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if Protocol_Type > 1.5
                        dict['Protocol_Type'] = Protocol_Type
                        threshholds.append(['Protocol_Type', '>', 1.5])
                        if Srate <= 1.44:
                            dict['Srate'] = Srate
                            threshholds.append(['Srate', '<=', 1.44])
                            if Tot_sum <= 1404.5:
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '<=', 1404.5])
                                return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Tot_sum > 1404.5
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '>', 1404.5])
                                return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                        else:  # if Srate > 1.44
                            dict['Srate'] = Srate
                            threshholds.append(['Srate', '>', 1.44])
                            if Max <= 362.18:
                                dict['Max'] = Max
                                threshholds.append(['Max', '<=', 362.18])
                                return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Max > 362.18
                                dict['Max'] = Max
                                threshholds.append(['Max', '>', 362.18])
                                return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                else:  # if TCP > 0.5
                    dict['TCP'] = TCP
                    threshholds.append(['TCP', '>', 0.5])
                    if Duration <= 60.77:
                        dict['Duration'] = Duration
                        threshholds.append(['Duration', '<=', 60.77])
                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                    else:  # if Duration > 60.77
                        dict['Duration'] = Duration
                        threshholds.append(['Duration', '>', 60.77])
                        if fin_flag_number <= 0.5:
                            dict['fin_flag_number'] = fin_flag_number
                            threshholds.append(['fin_flag_number', '<=', 0.5])
                            if IAT <= 83009724.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 83009724.0])
                                if Weight <= 135.2:
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '<=', 135.2])
                                    if Number <= 6.81:
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '<=', 6.81])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if Number > 6.81
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '>', 6.81])
                                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Weight > 135.2
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '>', 135.2])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if IAT > 83009724.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 83009724.0])
                                if Variance <= 0.3:
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '<=', 0.3])
                                    if rst_flag_number <= 0.5:
                                        dict['rst_flag_number'] = rst_flag_number
                                        threshholds.append(['rst_flag_number', '<=', 0.5])
                                        return [[0.00000000e+00, 0.00000000e+00, 9.99137931e-01, 0.00000000e+00,  0.00000000e+00, 8.62068966e-04, 0.00000000e+00, 0.00000000e+00]], dict, threshholds
                                    else:  # if rst_flag_number > 0.5
                                        dict['rst_flag_number'] = rst_flag_number
                                        threshholds.append(['rst_flag_number', '>', 0.5])
                                        return [[0. , 0. , 0.1, 0. , 0. , 0.9, 0. , 0. ]], dict, threshholds
                                else:  # if Variance > 0.3
                                    dict['Variance'] = Variance
                                    threshholds.append(['Variance', '>', 0.3])
                                    if fin_count <= 0.15:
                                        dict['fin_count'] = fin_count
                                        threshholds.append(['fin_count', '<=', 0.15])
                                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if fin_count > 0.15
                                        dict['fin_count'] = fin_count
                                        threshholds.append(['fin_count', '>', 0.15])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                        else:  # if fin_flag_number > 0.5
                            dict['fin_flag_number'] = fin_flag_number
                            threshholds.append(['fin_flag_number', '>', 0.5])
                            return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
            else:  # if Header_Length > 79.93
                dict['Header_Length'] = Header_Length
                threshholds.append(['Header_Length', '>', 79.93])
                if Magnitue <= 11.42:
                    dict['Magnitue'] = Magnitue
                    threshholds.append(['Magnitue', '<=', 11.42])
                    if Number <= 7.33:
                        dict['Number'] = Number
                        threshholds.append(['Number', '<=', 7.33])
                        if urg_count <= 0.25:
                            dict['urg_count'] = urg_count
                            threshholds.append(['urg_count', '<=', 0.25])
                            return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                        else:  # if urg_count > 0.25
                            dict['urg_count'] = urg_count
                            threshholds.append(['urg_count', '>', 0.25])
                            if fin_count <= 0.05:
                                dict['fin_count'] = fin_count
                                threshholds.append(['fin_count', '<=', 0.05])
                                if syn_flag_number <= 0.5:
                                    dict['syn_flag_number'] = syn_flag_number
                                    threshholds.append(['syn_flag_number', '<=', 0.5])
                                    if syn_count <= 1.3:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 1.3])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if syn_count > 1.3
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 1.3])
                                        return [[0.04, 0.44, 0.  , 0.  , 0.  , 0.04, 0.12, 0.36]], dict, threshholds
                                else:  # if syn_flag_number > 0.5
                                    dict['syn_flag_number'] = syn_flag_number
                                    threshholds.append(['syn_flag_number', '>', 0.5])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if fin_count > 0.05
                                dict['fin_count'] = fin_count
                                threshholds.append(['fin_count', '>', 0.05])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                    else:  # if Number > 7.33
                        dict['Number'] = Number
                        threshholds.append(['Number', '>', 7.33])
                        if IAT <= 83033560.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 83033560.0])
                            if Tot_size <= 50.01:
                                dict['Tot_size'] = Tot_size
                                threshholds.append(['Tot_size', '<=', 50.01])
                                if Number <= 9.33:
                                    dict['Number'] = Number
                                    threshholds.append(['Number', '<=', 9.33])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Number > 9.33
                                    dict['Number'] = Number
                                    threshholds.append(['Number', '>', 9.33])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Tot_size > 50.01
                                dict['Tot_size'] = Tot_size
                                threshholds.append(['Tot_size', '>', 50.01])
                                return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if IAT > 83033560.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 83033560.0])
                            if Duration <= 59.44:
                                dict['Duration'] = Duration
                                threshholds.append(['Duration', '<=', 59.44])
                                if urg_count <= 17.6:
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '<=', 17.6])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if urg_count > 17.6
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '>', 17.6])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if Duration > 59.44
                                dict['Duration'] = Duration
                                threshholds.append(['Duration', '>', 59.44])
                                if Weight <= 193.08:
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '<=', 193.08])
                                    return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Weight > 193.08
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '>', 193.08])
                                    if rst_count <= 105.55:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 105.55])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.96428571,  0.03571429, 0.   ]], dict, threshholds
                                    else:  # if rst_count > 105.55
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 105.55])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                else:  # if Magnitue > 11.42
                    dict['Magnitue'] = Magnitue
                    threshholds.append(['Magnitue', '>', 11.42])
                    if Magnitue <= 16.52:
                        dict['Magnitue'] = Magnitue
                        threshholds.append(['Magnitue', '<=', 16.52])
                        if IAT <= 0.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 0.0])
                            if Srate <= 364.06:
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '<=', 364.06])
                                if syn_count <= 0.1:
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '<=', 0.1])
                                    if Magnitue <= 11.48:
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '<=', 11.48])
                                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if Magnitue > 11.48
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '>', 11.48])
                                        return [[0.05797101, 0.   , 0.   , 0.   , 0.   , 0.01449275,  0.92753623, 0.   ]], dict, threshholds
                                else:  # if syn_count > 0.1
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '>', 0.1])
                                    if HTTPS <= 0.5:
                                        dict['HTTPS'] = HTTPS
                                        threshholds.append(['HTTPS', '<=', 0.5])
                                        return [[0.  , 0.25, 0.  , 0.  , 0.  , 0.5 , 0.  , 0.25]], dict, threshholds
                                    else:  # if HTTPS > 0.5
                                        dict['HTTPS'] = HTTPS
                                        threshholds.append(['HTTPS', '>', 0.5])
                                        return [[0.17241379, 0.05172414, 0.   , 0.   , 0.   , 0.25862069,  0.17241379, 0.34482759]], dict, threshholds
                            else:  # if Srate > 364.06
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '>', 364.06])
                                if AVG <= 65.98:
                                    dict['AVG'] = AVG
                                    threshholds.append(['AVG', '<=', 65.98])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if AVG > 65.98
                                    dict['AVG'] = AVG
                                    threshholds.append(['AVG', '>', 65.98])
                                    if syn_count <= 1.3:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 1.3])
                                        return [[0.01796407, 0.   , 0.   , 0.   , 0.   , 0.   ,  0.98203593, 0.   ]], dict, threshholds
                                    else:  # if syn_count > 1.3
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 1.3])
                                        return [[0.05555556, 0.11111111, 0.   , 0.   , 0.   , 0.05555556,  0.61111111, 0.16666667]], dict, threshholds
                        else:  # if IAT > 0.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 0.0])
                            if ack_flag_number <= 0.5:
                                dict['ack_flag_number'] = ack_flag_number
                                threshholds.append(['ack_flag_number', '<=', 0.5])
                                if Min <= 84.4:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 84.4])
                                    if Number <= 7.5:
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '<=', 7.5])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if Number > 7.5
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '>', 7.5])
                                        return [[0.   , 0.   , 0.08510638, 0.90425532, 0.   , 0.0106383 ,  0.   , 0.   ]], dict, threshholds
                                else:  # if Min > 84.4
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 84.4])
                                    if urg_count <= 0.0:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 0.0])
                                        return [[0.   , 0.   , 0.   , 0.125, 0.   , 0.   , 0.875, 0.   ]], dict, threshholds
                                    else:  # if urg_count > 0.0
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 0.0])
                                        return [[0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ]], dict, threshholds
                            else:  # if ack_flag_number > 0.5
                                dict['ack_flag_number'] = ack_flag_number
                                threshholds.append(['ack_flag_number', '>', 0.5])
                                if Weight <= 193.08:
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '<=', 193.08])
                                    if ack_count <= 0.3:
                                        dict['ack_count'] = ack_count
                                        threshholds.append(['ack_count', '<=', 0.3])
                                        return [[0.5255102 , 0.06887755, 0.00510204, 0.00765306, 0.   , 0.08163265,  0.20153061, 0.10969388]], dict, threshholds
                                    else:  # if ack_count > 0.3
                                        dict['ack_count'] = ack_count
                                        threshholds.append(['ack_count', '>', 0.3])
                                        return [[0.03448276, 0.4137931 , 0.   , 0.   , 0.   , 0.44827586,  0.06896552, 0.03448276]], dict, threshholds
                                else:  # if Weight > 193.08
                                    dict['Weight'] = Weight
                                    threshholds.append(['Weight', '>', 193.08])
                                    if Min <= 71.4:
                                        dict['Min'] = Min
                                        threshholds.append(['Min', '<=', 71.4])
                                        return [[0.14857143, 0.02857143, 0.   , 0.   , 0.   , 0.12571429,  0.51428571, 0.18285714]], dict, threshholds
                                    else:  # if Min > 71.4
                                        dict['Min'] = Min
                                        threshholds.append(['Min', '>', 71.4])
                                        return [[0.04347826, 0.   , 0.   , 0.   , 0.   , 0.   ,  0.95652174, 0.   ]], dict, threshholds
                    else:  # if Magnitue > 16.52
                        dict['Magnitue'] = Magnitue
                        threshholds.append(['Magnitue', '>', 16.52])
                        if Protocol_Type <= 6.83:
                            dict['Protocol_Type'] = Protocol_Type
                            threshholds.append(['Protocol_Type', '<=', 6.83])
                            if Header_Length <= 8176.57:
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '<=', 8176.57])
                                return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                            else:  # if Header_Length > 8176.57
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '>', 8176.57])
                                if Srate <= 232.33:
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '<=', 232.33])
                                    if Std <= 283.93:
                                        dict['Std'] = Std
                                        threshholds.append(['Std', '<=', 283.93])
                                        return [[0.   , 0.   , 0.   , 0.33333333, 0.   , 0.33333333,  0.   , 0.33333333]], dict, threshholds
                                    else:  # if Std > 283.93
                                        dict['Std'] = Std
                                        threshholds.append(['Std', '>', 283.93])
                                        return [[0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  ]], dict, threshholds
                                else:  # if Srate > 232.33
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '>', 232.33])
                                    if Tot_size <= 354.9:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 354.9])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if Tot_size > 354.9
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 354.9])
                                        return [[0.09090909, 0.   , 0.   , 0.   , 0.   , 0.09090909,  0.81818182, 0.   ]], dict, threshholds
                        else:  # if Protocol_Type > 6.83
                            dict['Protocol_Type'] = Protocol_Type
                            threshholds.append(['Protocol_Type', '>', 6.83])
                            if Variance <= 0.2:
                                dict['Variance'] = Variance
                                threshholds.append(['Variance', '<=', 0.2])
                                if Tot_size <= 378.72:
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '<=', 378.72])
                                    if Number <= 7.27:
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '<=', 7.27])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if Number > 7.27
                                        dict['Number'] = Number
                                        threshholds.append(['Number', '>', 7.27])
                                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Tot_size > 378.72
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '>', 378.72])
                                    return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                            else:  # if Variance > 0.2
                                dict['Variance'] = Variance
                                threshholds.append(['Variance', '>', 0.2])
                                if Magnitue <= 26.36:
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '<=', 26.36])
                                    return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Magnitue > 26.36
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '>', 26.36])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
        else:  # if Tot_sum > 3517.28
            dict['Tot_sum'] = Tot_sum
            threshholds.append(['Tot_sum', '>', 3517.28])
            if Tot_size <= 898.52:
                dict['Tot_size'] = Tot_size
                threshholds.append(['Tot_size', '<=', 898.52])
                if Tot_sum <= 3874.0:
                    dict['Tot_sum'] = Tot_sum
                    threshholds.append(['Tot_sum', '<=', 3874.0])
                    if Tot_size <= 425.04:
                        dict['Tot_size'] = Tot_size
                        threshholds.append(['Tot_size', '<=', 425.04])
                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                    else:  # if Tot_size > 425.04
                        dict['Tot_size'] = Tot_size
                        threshholds.append(['Tot_size', '>', 425.04])
                        return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                else:  # if Tot_sum > 3874.0
                    dict['Tot_sum'] = Tot_sum
                    threshholds.append(['Tot_sum', '>', 3874.0])
                    if Covariance <= 661932.61:
                        dict['Covariance'] = Covariance
                        threshholds.append(['Covariance', '<=', 661932.61])
                        return [[0., 0., 0., 0., 1., 0., 0., 0.]], dict, threshholds
                    else:  # if Covariance > 661932.61
                        dict['Covariance'] = Covariance
                        threshholds.append(['Covariance', '>', 661932.61])
                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
            else:  # if Tot_size > 898.52
                dict['Tot_size'] = Tot_size
                threshholds.append(['Tot_size', '>', 898.52])
                if TCP <= 0.5:
                    dict['TCP'] = TCP
                    threshholds.append(['TCP', '<=', 0.5])
                    if Max <= 1224.5:
                        dict['Max'] = Max
                        threshholds.append(['Max', '<=', 1224.5])
                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                    else:  # if Max > 1224.5
                        dict['Max'] = Max
                        threshholds.append(['Max', '>', 1224.5])
                        if Magnitue <= 49.94:
                            dict['Magnitue'] = Magnitue
                            threshholds.append(['Magnitue', '<=', 49.94])
                            return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                        else:  # if Magnitue > 49.94
                            dict['Magnitue'] = Magnitue
                            threshholds.append(['Magnitue', '>', 49.94])
                            if Srate <= 398.9:
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '<=', 398.9])
                                if Min <= 1342.0:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 1342.0])
                                    if flow_duration <= 0.57:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 0.57])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if flow_duration > 0.57
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 0.57])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                else:  # if Min > 1342.0
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 1342.0])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if Srate > 398.9
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '>', 398.9])
                                if Header_Length <= 901472.06:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 901472.06])
                                    if AVG <= 1338.72:
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '<=', 1338.72])
                                        return [[0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0.8]], dict, threshholds
                                    else:  # if AVG > 1338.72
                                        dict['AVG'] = AVG
                                        threshholds.append(['AVG', '>', 1338.72])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.33333333,  0.66666667, 0.   ]], dict, threshholds
                                else:  # if Header_Length > 901472.06
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 901472.06])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                else:  # if TCP > 0.5
                    dict['TCP'] = TCP
                    threshholds.append(['TCP', '>', 0.5])
                    if urg_count <= 891.1:
                        dict['urg_count'] = urg_count
                        threshholds.append(['urg_count', '<=', 891.1])
                        if HTTP <= 0.5:
                            dict['HTTP'] = HTTP
                            threshholds.append(['HTTP', '<=', 0.5])
                            if rst_count <= 465.5:
                                dict['rst_count'] = rst_count
                                threshholds.append(['rst_count', '<=', 465.5])
                                if psh_flag_number <= 0.5:
                                    dict['psh_flag_number'] = psh_flag_number
                                    threshholds.append(['psh_flag_number', '<=', 0.5])
                                    if Weight <= 141.55:
                                        dict['Weight'] = Weight
                                        threshholds.append(['Weight', '<=', 141.55])
                                        return [[0.07843137, 0.15686275, 0.   , 0.   , 0.   , 0.05882353,  0.07843137, 0.62745098]], dict, threshholds
                                    else:  # if Weight > 141.55
                                        dict['Weight'] = Weight
                                        threshholds.append(['Weight', '>', 141.55])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.02564103,  0.02564103, 0.94871795]], dict, threshholds
                                else:  # if psh_flag_number > 0.5
                                    dict['psh_flag_number'] = psh_flag_number
                                    threshholds.append(['psh_flag_number', '>', 0.5])
                                    if Srate <= 218.09:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 218.09])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.16666667,  0.16666667, 0.66666667]], dict, threshholds
                                    else:  # if Srate > 218.09
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 218.09])
                                        return [[0.1, 0. , 0. , 0. , 0. , 0.1, 0.7, 0.1]], dict, threshholds
                            else:  # if rst_count > 465.5
                                dict['rst_count'] = rst_count
                                threshholds.append(['rst_count', '>', 465.5])
                                if flow_duration <= 9.13:
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '<=', 9.13])
                                    if Tot_size <= 1599.65:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 1599.65])
                                        return [[0.10810811, 0.05405405, 0.   , 0.   , 0.   , 0.05405405,  0.13513514, 0.64864865]], dict, threshholds
                                    else:  # if Tot_size > 1599.65
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 1599.65])
                                        return [[0.1875, 0.    , 0.    , 0.    , 0.    , 0.    , 0.8125, 0.    ]], dict, threshholds
                                else:  # if flow_duration > 9.13
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '>', 9.13])
                                    if Srate <= 11.33:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 11.33])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                    else:  # if Srate > 11.33
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 11.33])
                                        return [[0.82352941, 0.01960784, 0.   , 0.   , 0.   , 0.05882353,  0.03921569, 0.05882353]], dict, threshholds
                        else:  # if HTTP > 0.5
                            dict['HTTP'] = HTTP
                            threshholds.append(['HTTP', '>', 0.5])
                            if AVG <= 2934.0:
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '<=', 2934.0])
                                if Srate <= 2289.03:
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '<=', 2289.03])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if Srate > 2289.03
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '>', 2289.03])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if AVG > 2934.0
                                dict['AVG'] = AVG
                                threshholds.append(['AVG', '>', 2934.0])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                    else:  # if urg_count > 891.1
                        dict['urg_count'] = urg_count
                        threshholds.append(['urg_count', '>', 891.1])
                        if Min <= 1359.2:
                            dict['Min'] = Min
                            threshholds.append(['Min', '<=', 1359.2])
                            return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                        else:  # if Min > 1359.2
                            dict['Min'] = Min
                            threshholds.append(['Min', '>', 1359.2])
                            return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
    else:  # if Variance > 0.39
        dict['Variance'] = Variance
        threshholds.append(['Variance', '>', 0.39])
        if IAT <= 166602848.0:
            dict['IAT'] = IAT
            threshholds.append(['IAT', '<=', 166602848.0])
            if IAT <= 166551400.0:
                dict['IAT'] = IAT
                threshholds.append(['IAT', '<=', 166551400.0])
                if ack_flag_number <= 0.5:
                    dict['ack_flag_number'] = ack_flag_number
                    threshholds.append(['ack_flag_number', '<=', 0.5])
                    if TCP <= 0.5:
                        dict['TCP'] = TCP
                        threshholds.append(['TCP', '<=', 0.5])
                        if Tot_size <= 286.65:
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '<=', 286.65])
                            if IAT <= 166499176.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 166499176.0])
                                if Header_Length <= 114323.2:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 114323.2])
                                    if Variance <= 0.91:
                                        dict['Variance'] = Variance
                                        threshholds.append(['Variance', '<=', 0.91])
                                        return [[0.09892473, 0.2562724 , 0.00179211, 0.01397849, 0.   , 0.14193548,  0.11326165, 0.37383513]], dict, threshholds
                                    else:  # if Variance > 0.91
                                        dict['Variance'] = Variance
                                        threshholds.append(['Variance', '>', 0.91])
                                        return [[0.   , 0.   , 0.   , 0.00548446, 0.023766  , 0.5630713 ,  0.   , 0.40767824]], dict, threshholds
                                else:  # if Header_Length > 114323.2
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 114323.2])
                                    if urg_count <= 20.65:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 20.65])
                                        return [[0.06130268, 0.   , 0.   , 0.00383142, 0.00766284, 0.02298851,  0.88122605, 0.02298851]], dict, threshholds
                                    else:  # if urg_count > 20.65
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 20.65])
                                        return [[0.45390071, 0.07801418, 0.   , 0.0141844 , 0.   , 0.21276596,  0.17730496, 0.06382979]], dict, threshholds
                            else:  # if IAT > 166499176.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 166499176.0])
                                return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                        else:  # if Tot_size > 286.65
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '>', 286.65])
                            if Number <= 7.15:
                                dict['Number'] = Number
                                threshholds.append(['Number', '<=', 7.15])
                                if Protocol_Type <= 13.95:
                                    dict['Protocol_Type'] = Protocol_Type
                                    threshholds.append(['Protocol_Type', '<=', 13.95])
                                    if flow_duration <= 74.14:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 74.14])
                                        return [[0.32110092, 0.03669725, 0.00917431, 0.   , 0.   , 0.09174312,  0.43119266, 0.11009174]], dict, threshholds
                                    else:  # if flow_duration > 74.14
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 74.14])
                                        return [[0.03571429, 0.35714286, 0.   , 0.   , 0.   , 0.10714286,  0.28571429, 0.21428571]], dict, threshholds
                                else:  # if Protocol_Type > 13.95
                                    dict['Protocol_Type'] = Protocol_Type
                                    threshholds.append(['Protocol_Type', '>', 13.95])
                                    if Duration <= 34.8:
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '<=', 34.8])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                    else:  # if Duration > 34.8
                                        dict['Duration'] = Duration
                                        threshholds.append(['Duration', '>', 34.8])
                                        return [[0.00940439, 0.0031348 , 0.   , 0.   , 0.   , 0.   ,  0.96238245, 0.02507837]], dict, threshholds
                            else:  # if Number > 7.15
                                dict['Number'] = Number
                                threshholds.append(['Number', '>', 7.15])
                                if Covariance <= 209448.2:
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '<=', 209448.2])
                                    if Variance <= 0.97:
                                        dict['Variance'] = Variance
                                        threshholds.append(['Variance', '<=', 0.97])
                                        return [[0.   , 0.   , 0.00469484, 0.00469484, 0.98591549, 0.00469484,  0.   , 0.   ]], dict, threshholds
                                    else:  # if Variance > 0.97
                                        dict['Variance'] = Variance
                                        threshholds.append(['Variance', '>', 0.97])
                                        return [[0.48648649, 0.   , 0.   , 0.   , 0.   , 0.45945946,  0.   , 0.05405405]], dict, threshholds
                                else:  # if Covariance > 209448.2
                                    dict['Covariance'] = Covariance
                                    threshholds.append(['Covariance', '>', 209448.2])
                                    if Weight <= 193.08:
                                        dict['Weight'] = Weight
                                        threshholds.append(['Weight', '<=', 193.08])
                                        return [[0.   , 0.   , 0.94512195, 0.01829268, 0.03658537, 0.   ,  0.   , 0.   ]], dict, threshholds
                                    else:  # if Weight > 193.08
                                        dict['Weight'] = Weight
                                        threshholds.append(['Weight', '>', 193.08])
                                        return [[0.71428571, 0.   , 0.   , 0.   , 0.   , 0.2  ,  0.   , 0.08571429]], dict, threshholds
                    else:  # if TCP > 0.5
                        dict['TCP'] = TCP
                        threshholds.append(['TCP', '>', 0.5])
                        if Tot_size <= 68.7:
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '<=', 68.7])
                            if Srate <= 9.22:
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '<=', 9.22])
                                if syn_count <= 1.19:
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '<=', 1.19])
                                    if Max <= 56.21:
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '<=', 56.21])
                                        return [[0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ]], dict, threshholds
                                    else:  # if Max > 56.21
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '>', 56.21])
                                        return [[0.   , 0.04651163, 0.02325581, 0.04651163, 0.   , 0.86046512,  0.   , 0.02325581]], dict, threshholds
                                else:  # if syn_count > 1.19
                                    dict['syn_count'] = syn_count
                                    threshholds.append(['syn_count', '>', 1.19])
                                    if rst_count <= 4.57:
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '<=', 4.57])
                                        return [[0., 0., 0., 1., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if rst_count > 4.57
                                        dict['rst_count'] = rst_count
                                        threshholds.append(['rst_count', '>', 4.57])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if Srate > 9.22
                                dict['Srate'] = Srate
                                threshholds.append(['Srate', '>', 9.22])
                                if Min <= 47.0:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 47.0])
                                    return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if Min > 47.0
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 47.0])
                                    if syn_count <= 1.0:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 1.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.99754902,  0.00245098, 0.   ]], dict, threshholds
                                    else:  # if syn_count > 1.0
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 1.0])
                                        return [[0.1, 0.1, 0. , 0.3, 0. , 0.5, 0. , 0. ]], dict, threshholds
                        else:  # if Tot_size > 68.7
                            dict['Tot_size'] = Tot_size
                            threshholds.append(['Tot_size', '>', 68.7])
                            if syn_flag_number <= 0.5:
                                dict['syn_flag_number'] = syn_flag_number
                                threshholds.append(['syn_flag_number', '<=', 0.5])
                                if Tot_sum <= 936.3:
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '<=', 936.3])
                                    if Header_Length <= 50502.1:
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '<=', 50502.1])
                                        return [[0.06270627, 0.20462046, 0.00660066, 0.01320132, 0.   , 0.39273927,  0.08250825, 0.23762376]], dict, threshholds
                                    else:  # if Header_Length > 50502.1
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '>', 50502.1])
                                        return [[0.38095238, 0.19047619, 0.   , 0.   , 0.   , 0.35714286,  0.04761905, 0.02380952]], dict, threshholds
                                else:  # if Tot_sum > 936.3
                                    dict['Tot_sum'] = Tot_sum
                                    threshholds.append(['Tot_sum', '>', 936.3])
                                    if Tot_size <= 417.53:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 417.53])
                                        return [[0.088, 0.028, 0.004, 0.048, 0.004, 0.624, 0.008, 0.196]], dict, threshholds
                                    else:  # if Tot_size > 417.53
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 417.53])
                                        return [[0.25 , 0.   , 0.21428571, 0.17857143, 0.   , 0.17857143,  0.10714286, 0.07142857]], dict, threshholds
                            else:  # if syn_flag_number > 0.5
                                dict['syn_flag_number'] = syn_flag_number
                                threshholds.append(['syn_flag_number', '>', 0.5])
                                if Tot_size <= 121.15:
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '<=', 121.15])
                                    if Tot_size <= 107.36:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 107.36])
                                        return [[0.   , 0.01886792, 0.   , 0.   , 0.   , 0.96226415,  0.01886792, 0.   ]], dict, threshholds
                                    else:  # if Tot_size > 107.36
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 107.36])
                                        return [[0.   , 0.   , 0.33333333, 0.   , 0.   , 0.66666667,  0.   , 0.   ]], dict, threshholds
                                else:  # if Tot_size > 121.15
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '>', 121.15])
                                    if Tot_size <= 153.2:
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '<=', 153.2])
                                        return [[0.   , 0.   , 0.83333333, 0.16666667, 0.   , 0.   ,  0.   , 0.   ]], dict, threshholds
                                    else:  # if Tot_size > 153.2
                                        dict['Tot_size'] = Tot_size
                                        threshholds.append(['Tot_size', '>', 153.2])
                                        return [[0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. ]], dict, threshholds
                else:  # if ack_flag_number > 0.5
                    dict['ack_flag_number'] = ack_flag_number
                    threshholds.append(['ack_flag_number', '>', 0.5])
                    if IAT <= 166499168.0:
                        dict['IAT'] = IAT
                        threshholds.append(['IAT', '<=', 166499168.0])
                        if IAT <= 83199244.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 83199244.0])
                            if Header_Length <= 164437.34:
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '<=', 164437.34])
                                if fin_count <= 0.49:
                                    dict['fin_count'] = fin_count
                                    threshholds.append(['fin_count', '<=', 0.49])
                                    if SSH <= 0.5:
                                        dict['SSH'] = SSH
                                        threshholds.append(['SSH', '<=', 0.5])
                                        return [[0.13205467, 0.31305115, 0.00088183, 0.00595238, 0.   , 0.16776896,  0.10471781, 0.27557319]], dict, threshholds
                                    else:  # if SSH > 0.5
                                        dict['SSH'] = SSH
                                        threshholds.append(['SSH', '>', 0.5])
                                        return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
                                else:  # if fin_count > 0.49
                                    dict['fin_count'] = fin_count
                                    threshholds.append(['fin_count', '>', 0.49])
                                    if Tot_sum <= 4667.86:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 4667.86])
                                        return [[0.0125523 , 0.041841  , 0.   , 0.0167364 , 0.   , 0.89539749,  0.0292887 , 0.0041841 ]], dict, threshholds
                                    else:  # if Tot_sum > 4667.86
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 4667.86])
                                        return [[0.    , 0.    , 0.0625, 0.875 , 0.    , 0.0625, 0.    , 0.    ]], dict, threshholds
                            else:  # if Header_Length > 164437.34
                                dict['Header_Length'] = Header_Length
                                threshholds.append(['Header_Length', '>', 164437.34])
                                if IAT <= 0.0:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 0.0])
                                    if IAT <= 0.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 0.0])
                                        return [[0.26031294, 0.0227596 , 0.   , 0.   , 0.   , 0.02702703,  0.6002845 , 0.08961593]], dict, threshholds
                                    else:  # if IAT > 0.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 0.0])
                                        return [[0.45631068, 0.04004854, 0.   , 0.   , 0.   , 0.07281553,  0.36407767, 0.06674757]], dict, threshholds
                                else:  # if IAT > 0.0
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 0.0])
                                    if syn_count <= 0.05:
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '<=', 0.05])
                                        return [[0.56512141, 0.02428256, 0.   , 0.00220751, 0.   , 0.04415011,  0.33333333, 0.03090508]], dict, threshholds
                                    else:  # if syn_count > 0.05
                                        dict['syn_count'] = syn_count
                                        threshholds.append(['syn_count', '>', 0.05])
                                        return [[0.70373921, 0.06232023, 0.   , 0.00191755, 0.   , 0.1093001 ,  0.0949185 , 0.02780441]], dict, threshholds
                        else:  # if IAT > 83199244.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 83199244.0])
                            if Variance <= 0.97:
                                dict['Variance'] = Variance
                                threshholds.append(['Variance', '<=', 0.97])
                                if Magnitue <= 32.78:
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '<=', 32.78])
                                    if HTTPS <= 0.5:
                                        dict['HTTPS'] = HTTPS
                                        threshholds.append(['HTTPS', '<=', 0.5])
                                        return [[0.   , 0.   , 0.0109589 , 0.   , 0.0109589 , 0.97808219,  0.   , 0.   ]], dict, threshholds
                                    else:  # if HTTPS > 0.5
                                        dict['HTTPS'] = HTTPS
                                        threshholds.append(['HTTPS', '>', 0.5])
                                        return [[0.   , 0.   , 0.0212766 , 0.   , 0.08510638, 0.89361702,  0.   , 0.   ]], dict, threshholds
                                else:  # if Magnitue > 32.78
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '>', 32.78])
                                    if Header_Length <= 6183.24:
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '<=', 6183.24])
                                        return [[0., 0., 1., 0., 0., 0., 0., 0.]], dict, threshholds
                                    else:  # if Header_Length > 6183.24
                                        dict['Header_Length'] = Header_Length
                                        threshholds.append(['Header_Length', '>', 6183.24])
                                        return [[0.   , 0.   , 0.01515152, 0.   , 0.15151515, 0.83333333,  0.   , 0.   ]], dict, threshholds
                            else:  # if Variance > 0.97
                                dict['Variance'] = Variance
                                threshholds.append(['Variance', '>', 0.97])
                                if rst_count <= 274.95:
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '<=', 274.95])
                                    if urg_count <= 0.55:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 0.55])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.98461538,  0.   , 0.01538462]], dict, threshholds
                                    else:  # if urg_count > 0.55
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 0.55])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.58986731,  0.   , 0.41013269]], dict, threshholds
                                else:  # if rst_count > 274.95
                                    dict['rst_count'] = rst_count
                                    threshholds.append(['rst_count', '>', 274.95])
                                    if psh_flag_number <= 0.5:
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '<=', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.90789474,  0.   , 0.09210526]], dict, threshholds
                                    else:  # if psh_flag_number > 0.5
                                        dict['psh_flag_number'] = psh_flag_number
                                        threshholds.append(['psh_flag_number', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                    else:  # if IAT > 166499168.0
                        dict['IAT'] = IAT
                        threshholds.append(['IAT', '>', 166499168.0])
                        return [[1., 0., 0., 0., 0., 0., 0., 0.]], dict, threshholds
            else:  # if IAT > 166551400.0
                dict['IAT'] = IAT
                threshholds.append(['IAT', '>', 166551400.0])
                return [[0., 1., 0., 0., 0., 0., 0., 0.]], dict, threshholds
        else:  # if IAT > 166602848.0
            dict['IAT'] = IAT
            threshholds.append(['IAT', '>', 166602848.0])
            if Tot_sum <= 1094.35:
                dict['Tot_sum'] = Tot_sum
                threshholds.append(['Tot_sum', '<=', 1094.35])
                if IAT <= 166850792.0:
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '<=', 166850792.0])
                    if Header_Length <= 25548.55:
                        dict['Header_Length'] = Header_Length
                        threshholds.append(['Header_Length', '<=', 25548.55])
                        if IAT <= 166846200.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 166846200.0])
                            return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                        else:  # if IAT > 166846200.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 166846200.0])
                            return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                    else:  # if Header_Length > 25548.55
                        dict['Header_Length'] = Header_Length
                        threshholds.append(['Header_Length', '>', 25548.55])
                        if syn_count <= 0.9:
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '<=', 0.9])
                            if Tot_size <= 88.65:
                                dict['Tot_size'] = Tot_size
                                threshholds.append(['Tot_size', '<=', 88.65])
                                return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                            else:  # if Tot_size > 88.65
                                dict['Tot_size'] = Tot_size
                                threshholds.append(['Tot_size', '>', 88.65])
                                if Srate <= 30.04:
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '<=', 30.04])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if Srate > 30.04
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '>', 30.04])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                        else:  # if syn_count > 0.9
                            dict['syn_count'] = syn_count
                            threshholds.append(['syn_count', '>', 0.9])
                            if urg_count <= 6.1:
                                dict['urg_count'] = urg_count
                                threshholds.append(['urg_count', '<=', 6.1])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if urg_count > 6.1
                                dict['urg_count'] = urg_count
                                threshholds.append(['urg_count', '>', 6.1])
                                if Magnitue <= 11.71:
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '<=', 11.71])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if Magnitue > 11.71
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '>', 11.71])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                else:  # if IAT > 166850792.0
                    dict['IAT'] = IAT
                    threshholds.append(['IAT', '>', 166850792.0])
                    if IAT <= 167246296.0:
                        dict['IAT'] = IAT
                        threshholds.append(['IAT', '<=', 167246296.0])
                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                    else:  # if IAT > 167246296.0
                        dict['IAT'] = IAT
                        threshholds.append(['IAT', '>', 167246296.0])
                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
            else:  # if Tot_sum > 1094.35
                dict['Tot_sum'] = Tot_sum
                threshholds.append(['Tot_sum', '>', 1094.35])
                if Rate <= 35.62:
                    dict['Rate'] = Rate
                    threshholds.append(['Rate', '<=', 35.62])
                    if HTTPS <= 0.5:
                        dict['HTTPS'] = HTTPS
                        threshholds.append(['HTTPS', '<=', 0.5])
                        if Min <= 58.5:
                            dict['Min'] = Min
                            threshholds.append(['Min', '<=', 58.5])
                            if syn_flag_number <= 0.5:
                                dict['syn_flag_number'] = syn_flag_number
                                threshholds.append(['syn_flag_number', '<=', 0.5])
                                if Radius <= 95.15:
                                    dict['Radius'] = Radius
                                    threshholds.append(['Radius', '<=', 95.15])
                                    if Tot_sum <= 1252.5:
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '<=', 1252.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.43333333,  0.45 , 0.11666667]], dict, threshholds
                                    else:  # if Tot_sum > 1252.5
                                        dict['Tot_sum'] = Tot_sum
                                        threshholds.append(['Tot_sum', '>', 1252.5])
                                        return [[0.   , 0.00188679, 0.   , 0.   , 0.   , 0.33773585,  0.20188679, 0.45849057]], dict, threshholds
                                else:  # if Radius > 95.15
                                    dict['Radius'] = Radius
                                    threshholds.append(['Radius', '>', 95.15])
                                    if Srate <= 22.41:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 22.41])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.18547141,  0.20092736, 0.61360124]], dict, threshholds
                                    else:  # if Srate > 22.41
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 22.41])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.05479452,  0.60273973, 0.34246575]], dict, threshholds
                            else:  # if syn_flag_number > 0.5
                                dict['syn_flag_number'] = syn_flag_number
                                threshholds.append(['syn_flag_number', '>', 0.5])
                                return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                        else:  # if Min > 58.5
                            dict['Min'] = Min
                            threshholds.append(['Min', '>', 58.5])
                            if rst_count <= 0.25:
                                dict['rst_count'] = rst_count
                                threshholds.append(['rst_count', '<=', 0.25])
                                if Magnitue <= 15.1:
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '<=', 15.1])
                                    if DNS <= 0.5:
                                        dict['DNS'] = DNS
                                        threshholds.append(['DNS', '<=', 0.5])
                                        return [[0. , 0. , 0. , 0. , 0. , 0.2, 0.8, 0. ]], dict, threshholds
                                    else:  # if DNS > 0.5
                                        dict['DNS'] = DNS
                                        threshholds.append(['DNS', '>', 0.5])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if Magnitue > 15.1
                                    dict['Magnitue'] = Magnitue
                                    threshholds.append(['Magnitue', '>', 15.1])
                                    if flow_duration <= 12.26:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 12.26])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.88888889, 0.11111111]], dict, threshholds
                                    else:  # if flow_duration > 12.26
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 12.26])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.35, 0.6 ]], dict, threshholds
                            else:  # if rst_count > 0.25
                                dict['rst_count'] = rst_count
                                threshholds.append(['rst_count', '>', 0.25])
                                if Header_Length <= 66870.45:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 66870.45])
                                    if Radius <= 50.4:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 50.4])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if Radius > 50.4
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 50.4])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.06666667,  0.18809524, 0.7452381 ]], dict, threshholds
                                else:  # if Header_Length > 66870.45
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 66870.45])
                                    if urg_count <= 51.05:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 51.05])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.85, 0.15]], dict, threshholds
                                    else:  # if urg_count > 51.05
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 51.05])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.075, 0.25 , 0.675]], dict, threshholds
                    else:  # if HTTPS > 0.5
                        dict['HTTPS'] = HTTPS
                        threshholds.append(['HTTPS', '>', 0.5])
                        if Std <= 670.22:
                            dict['Std'] = Std
                            threshholds.append(['Std', '<=', 670.22])
                            if Std <= 70.12:
                                dict['Std'] = Std
                                threshholds.append(['Std', '<=', 70.12])
                                if Min <= 59.8:
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '<=', 59.8])
                                    if urg_count <= 179.1:
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '<=', 179.1])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.28571429,  0.27142857, 0.44285714]], dict, threshholds
                                    else:  # if urg_count > 179.1
                                        dict['urg_count'] = urg_count
                                        threshholds.append(['urg_count', '>', 179.1])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.82758621,  0.17241379, 0.   ]], dict, threshholds
                                else:  # if Min > 59.8
                                    dict['Min'] = Min
                                    threshholds.append(['Min', '>', 59.8])
                                    if flow_duration <= 612.57:
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '<=', 612.57])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.0952381 ,  0.52380952, 0.38095238]], dict, threshholds
                                    else:  # if flow_duration > 612.57
                                        dict['flow_duration'] = flow_duration
                                        threshholds.append(['flow_duration', '>', 612.57])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                            else:  # if Std > 70.12
                                dict['Std'] = Std
                                threshholds.append(['Std', '>', 70.12])
                                if Header_Length <= 59558.6:
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '<=', 59558.6])
                                    if Rate <= 2.25:
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '<=', 2.25])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.11111111,  0.04444444, 0.84444444]], dict, threshholds
                                    else:  # if Rate > 2.25
                                        dict['Rate'] = Rate
                                        threshholds.append(['Rate', '>', 2.25])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.12931034,  0.24568966, 0.625     ]], dict, threshholds
                                else:  # if Header_Length > 59558.6
                                    dict['Header_Length'] = Header_Length
                                    threshholds.append(['Header_Length', '>', 59558.6])
                                    if IAT <= 167246408.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 167246408.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.02040816,  0.95918367, 0.02040816]], dict, threshholds
                                    else:  # if IAT > 167246408.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 167246408.0])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                        else:  # if Std > 670.22
                            dict['Std'] = Std
                            threshholds.append(['Std', '>', 670.22])
                            if Tot_sum <= 12664.95:
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '<=', 12664.95])
                                if urg_count <= 18.05:
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '<=', 18.05])
                                    if IAT <= 167249672.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 167249672.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.46153846,  0.15384615, 0.38461538]], dict, threshholds
                                    else:  # if IAT > 167249672.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 167249672.0])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if urg_count > 18.05
                                    dict['urg_count'] = urg_count
                                    threshholds.append(['urg_count', '>', 18.05])
                                    if Max <= 2938.0:
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '<=', 2938.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.94444444, 0.05555556]], dict, threshholds
                                    else:  # if Max > 2938.0
                                        dict['Max'] = Max
                                        threshholds.append(['Max', '>', 2938.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.14285714,  0.57142857, 0.28571429]], dict, threshholds
                            else:  # if Tot_sum > 12664.95
                                dict['Tot_sum'] = Tot_sum
                                threshholds.append(['Tot_sum', '>', 12664.95])
                                if Tot_size <= 94.3:
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '<=', 94.3])
                                    if Magnitue <= 47.94:
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '<=', 47.94])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if Magnitue > 47.94
                                        dict['Magnitue'] = Magnitue
                                        threshholds.append(['Magnitue', '>', 47.94])
                                        return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                                else:  # if Tot_size > 94.3
                                    dict['Tot_size'] = Tot_size
                                    threshholds.append(['Tot_size', '>', 94.3])
                                    if Radius <= 2461.64:
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '<=', 2461.64])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.05714286,  0.85714286, 0.08571429]], dict, threshholds
                                    else:  # if Radius > 2461.64
                                        dict['Radius'] = Radius
                                        threshholds.append(['Radius', '>', 2461.64])
                                        return [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.75]], dict, threshholds
                else:  # if Rate > 35.62
                    dict['Rate'] = Rate
                    threshholds.append(['Rate', '>', 35.62])
                    if urg_count <= 163.8:
                        dict['urg_count'] = urg_count
                        threshholds.append(['urg_count', '<=', 163.8])
                        if IAT <= 167246344.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 167246344.0])
                            if Duration <= 239.9:
                                dict['Duration'] = Duration
                                threshholds.append(['Duration', '<=', 239.9])
                                if flow_duration <= 1.56:
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '<=', 1.56])
                                    if Srate <= 742.53:
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '<=', 742.53])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.10571429,  0.57428571, 0.32 ]], dict, threshholds
                                    else:  # if Srate > 742.53
                                        dict['Srate'] = Srate
                                        threshholds.append(['Srate', '>', 742.53])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.01408451,  0.86619718, 0.11971831]], dict, threshholds
                                else:  # if flow_duration > 1.56
                                    dict['flow_duration'] = flow_duration
                                    threshholds.append(['flow_duration', '>', 1.56])
                                    if HTTP <= 0.5:
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '<=', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.04924875,  0.85475793, 0.09599332]], dict, threshholds
                                    else:  # if HTTP > 0.5
                                        dict['HTTP'] = HTTP
                                        threshholds.append(['HTTP', '>', 0.5])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.98461538, 0.01538462]], dict, threshholds
                            else:  # if Duration > 239.9
                                dict['Duration'] = Duration
                                threshholds.append(['Duration', '>', 239.9])
                                if Protocol_Type <= 8.75:
                                    dict['Protocol_Type'] = Protocol_Type
                                    threshholds.append(['Protocol_Type', '<=', 8.75])
                                    if IAT <= 166846232.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 166846232.0])
                                        return [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,  0.17391304, 0.82608696]], dict, threshholds
                                    else:  # if IAT > 166846232.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 166846232.0])
                                        return [[0.    , 0.    , 0.    , 0.    , 0.    , 0.8125, 0.1875, 0.    ]], dict, threshholds
                                else:  # if Protocol_Type > 8.75
                                    dict['Protocol_Type'] = Protocol_Type
                                    threshholds.append(['Protocol_Type', '>', 8.75])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                        else:  # if IAT > 167246344.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 167246344.0])
                            return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                    else:  # if urg_count > 163.8
                        dict['urg_count'] = urg_count
                        threshholds.append(['urg_count', '>', 163.8])
                        if IAT <= 166729208.0:
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '<=', 166729208.0])
                            return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                        else:  # if IAT > 166729208.0
                            dict['IAT'] = IAT
                            threshholds.append(['IAT', '>', 166729208.0])
                            if IAT <= 166851008.0:
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '<=', 166851008.0])
                                if IAT <= 166846248.0:
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '<=', 166846248.0])
                                    return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if IAT > 166846248.0
                                    dict['IAT'] = IAT
                                    threshholds.append(['IAT', '>', 166846248.0])
                                    return [[0., 0., 0., 0., 0., 1., 0., 0.]], dict, threshholds
                            else:  # if IAT > 166851008.0
                                dict['IAT'] = IAT
                                threshholds.append(['IAT', '>', 166851008.0])
                                if Srate <= 81.36:
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '<=', 81.36])
                                    if IAT <= 167241960.0:
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '<=', 167241960.0])
                                        return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds
                                    else:  # if IAT > 167241960.0
                                        dict['IAT'] = IAT
                                        threshholds.append(['IAT', '>', 167241960.0])
                                        return [[0., 0., 0., 0., 0., 0., 0., 1.]], dict, threshholds
                                else:  # if Srate > 81.36
                                    dict['Srate'] = Srate
                                    threshholds.append(['Srate', '>', 81.36])
                                    return [[0., 0., 0., 0., 0., 0., 1., 0.]], dict, threshholds

In [12]:
def get_prediction_label_1(row):
    prediction_probs, dict, thresholds = predict_1(*row[:-1])
    max_index = np.argmax(prediction_probs)
    return labels[max_index]

In [13]:
def get_prediction_prediction_accuracy_1(row):
    prediction_probs, results_dict, thresholds = predict_1(*row[:-2])
    # Define a mapping for names that differ in format
    name_mapping = {
        # 'Tot_sum': 'Tot sum',
        # 'Tot_size': 'Tot size',
        # 'Protocol_Type': 'Protocol Type'
    }
    
    # Convert thresholds into a readable string format with proper naming
    conditions_str = " and ".join([
        f"({name_mapping.get(condition[0], condition[0])} {condition[1]} {condition[2]})"
        for condition in thresholds
    ])
    search_string = f"if {conditions_str} then class:"

    # Read the file and search for the pattern
    with open('1_decision_tree_rule-9-natrual_language.txt', 'r') as file:
        for line in file:
            if search_string in line:
                # Extract probability
                prob_match = re.search(r"\(proba: (\d+\.\d+)%\)", line)
                if prob_match:
                    return float(prob_match.group(1))  # Return as float percentage
    #print(search_string)
    return None

In [14]:
def get_prediction_label_2(row):
    prediction_probs, dict, thresholds = predict_2(*row[:-3])
    max_index = np.argmax(prediction_probs)
    return labels[max_index]

In [15]:
def get_prediction_prediction_accuracy_2(row):
    prediction_probs, results_dict, thresholds = predict_2(*row[:-4])
    # Define a mapping for names that differ in format
    name_mapping = {
        # 'Tot_sum': 'Tot sum',
        # 'Tot_size': 'Tot size',
        # 'Protocol_Type': 'Protocol Type'
    }
    
    # Convert thresholds into a readable string format with proper naming
    conditions_str = " and ".join([
        f"({name_mapping.get(condition[0], condition[0])} {condition[1]} {condition[2]})"
        for condition in thresholds
    ])
    search_string = f"if {conditions_str} then class:"

    # Read the file and search for the pattern
    with open('2_decision_tree_rule-9-natrual_language.txt', 'r') as file:
        for line in file:
            if search_string in line:
                # Extract probability
                prob_match = re.search(r"\(proba: (\d+\.\d+)%\)", line)
                if prob_match:
                    return float(prob_match.group(1))  # Return as float percentage
    #print(search_string)
    return None

In [16]:
df['prediction_1']= df.apply(lambda row: get_prediction_label_1(row), axis=1)
df['accuracy_1'] = df.apply(lambda row: get_prediction_prediction_accuracy_1(row), axis=1)

In [17]:
df['prediction_2']= df.apply(lambda row: get_prediction_label_2(row), axis=1)
df['accuracy_2'] = df.apply(lambda row: get_prediction_prediction_accuracy_2(row), axis=1)

In [18]:
df.head()

,flow_duration,Header_Length,Protocol_Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot_sum,Min,Max,AVG,Std,Tot_size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label,prediction_1,accuracy_1,prediction_2,accuracy_2
0,13.215984,4.348996e+05,6.0,232.000000,62.537910,62.537910,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.543743,0.054555,32.0,338.06982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2029.0,66.000000,1072.000000,368.706430,201.429741,66.0,8.352267e+07,13.5,16.324726,354.298748,63225.672546,1.0,244.6,BenignTraffic,BenignTraffic,81.16,Recon,90.79
1,122.479718,6.217280e+04,13.7,86.901951,5.057926,5.057926,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058849,0.000000,0.000000,172.8,351.20000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1101.9,50.000000,230.000000,71.085046,45.831857,68.1,1.665238e+08,13.5,11.920195,64.970328,2114.415628,1.0,244.6,BenignTraffic,BenignTraffic,100.00,BenignTraffic,100.00
2,17.803338,4.348996e+05,7.1,83.100000,81.390655,81.390655,0.000002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,1.600000,0.054555,79.2,838.80000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,17328.5,54.000000,776.286826,368.706430,827.989774,1441.5,1.665213e+08,13.5,47.082245,1170.507395,688084.298393,1.0,244.6,BenignTraffic,BenignTraffic,81.16,BenignTraffic,100.00
3,24.131660,1.738870e+06,6.0,64.000000,70.136075,70.136075,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,2.000000,0.000000,64.1,1691.50000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,10183.0,180.739432,2672.400000,368.706430,505.204813,1777.2,2.089024e-04,5.5,60.131155,714.467498,320093.330391,0.8,38.5,BenignTraffic,BenignTraffic,70.74,BenignTraffic,70.37
4,178.343702,5.750180e+05,6.0,232.000000,316.569854,316.569854,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,2.000000,0.000000,17.0,338.06982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,363.0,66.000000,66.000000,66.000000,0.000000,66.0,8.352267e+07,5.5,11.489125,0.000000,205955.545490,0.0,38.5,BenignTraffic,Spoofing,100.00,BenignTraffic,52.55


In [19]:
def choose_higher_accuracy(row):
    if row['accuracy_1'] >= row['accuracy_2']:
        return row['prediction_1']
    else:
        return row['prediction_2']

In [20]:
df['predict_result'] = df.apply(choose_higher_accuracy, axis=1)

In [21]:
df.head()

,flow_duration,Header_Length,Protocol_Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot_sum,Min,Max,AVG,Std,Tot_size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label,prediction_1,accuracy_1,prediction_2,accuracy_2,predict_result
0,13.215984,4.348996e+05,6.0,232.000000,62.537910,62.537910,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.543743,0.054555,32.0,338.06982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2029.0,66.000000,1072.000000,368.706430,201.429741,66.0,8.352267e+07,13.5,16.324726,354.298748,63225.672546,1.0,244.6,BenignTraffic,BenignTraffic,81.16,Recon,90.79,Recon
1,122.479718,6.217280e+04,13.7,86.901951,5.057926,5.057926,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058849,0.000000,0.000000,172.8,351.20000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1101.9,50.000000,230.000000,71.085046,45.831857,68.1,1.665238e+08,13.5,11.920195,64.970328,2114.415628,1.0,244.6,BenignTraffic,BenignTraffic,100.00,BenignTraffic,100.00,BenignTraffic
2,17.803338,4.348996e+05,7.1,83.100000,81.390655,81.390655,0.000002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,1.600000,0.054555,79.2,838.80000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,17328.5,54.000000,776.286826,368.706430,827.989774,1441.5,1.665213e+08,13.5,47.082245,1170.507395,688084.298393,1.0,244.6,BenignTraffic,BenignTraffic,81.16,BenignTraffic,100.00,BenignTraffic
3,24.131660,1.738870e+06,6.0,64.000000,70.136075,70.136075,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,2.000000,0.000000,64.1,1691.50000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,10183.0,180.739432,2672.400000,368.706430,505.204813,1777.2,2.089024e-04,5.5,60.131155,714.467498,320093.330391,0.8,38.5,BenignTraffic,BenignTraffic,70.74,BenignTraffic,70.37,BenignTraffic
4,178.343702,5.750180e+05,6.0,232.000000,316.569854,316.569854,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,2.000000,0.000000,17.0,338.06982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,363.0,66.000000,66.000000,66.000000,0.000000,66.0,8.352267e+07,5.5,11.489125,0.000000,205955.545490,0.0,38.5,BenignTraffic,Spoofing,100.00,BenignTraffic,52.55,Spoofing


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [23]:
predictions = df['predict_result'] 
actual_labels = df['label']

# Calculate the metrics
accuracy = accuracy_score(actual_labels, predictions)
precision = precision_score(actual_labels, predictions, average='macro')  # Adjusted for multi-class
recall = recall_score(actual_labels, predictions, average='macro')  # Adjusted for multi-class
f1 = f1_score(actual_labels, predictions, average='macro')  # Adjusted for multi-class

print(f"Sampel size: {len(df):}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Sampel size: 5000
Accuracy: 0.66
Precision: 0.68
Recall: 0.66
F1 Score: 0.66


In [24]:
df.to_csv('baseline_file.csv', index=False)